In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
from tqdm import tqdm
import numpy as np
import torch
import h5py
import pandas as pd
import rdkit.Chem as Chem
from rdkit.Chem import Draw
import io
from PIL import Image
import hickle as hkl

In [ ]:
DELQSAR_ROOT = os.getcwd() + '/../../'
sys.path += [DELQSAR_ROOT + '/../']

from del_qsar import models, featurizers

if not os.path.isdir('substructure_pair_analysis'):
    os.mkdir('substructure_pair_analysis')
def pathify(fname):
    return os.path.join('substructure_pair_analysis', fname)

In [ ]:
DD1S_FINGERPRINTS_FILENAME = 'x_DD1S_CAIX_2048_bits_all_fps.h5' # should be in experiments folder
triazine_FINGERPRINTS_FILENAME = 'x_triazine_2048_bits_all_fps.h5' # should be in experiments folder
DD1S_CAIX_RANDOM_SPLIT_FP_FFNN_SEED_0_MODEL_PATH = os.path.join(DELQSAR_ROOT, 'experiments',
                                                  'models', 'DD1S_CAIX', 'FP-FFNN', 'random_seed_0.torch')
DD1S_CAIX_RANDOM_SPLIT_FP_FFNN_SEED_1_MODEL_PATH = os.path.join(DELQSAR_ROOT, 'experiments',
                                                  'models', 'DD1S_CAIX', 'FP-FFNN', 'random_seed_1.torch')
DD1S_CAIX_RANDOM_SPLIT_FP_FFNN_SEED_2_MODEL_PATH = os.path.join(DELQSAR_ROOT, 'experiments',
                                                  'models', 'DD1S_CAIX', 'FP-FFNN', 'random_seed_2.torch')
triazine_sEH_RANDOM_SPLIT_FP_FFNN_SEED_0_MODEL_PATH = os.path.join(DELQSAR_ROOT, 'experiments',
                                                  'models', 'triazine_sEH', 'FP-FFNN', 'random_seed_0.torch')
triazine_SIRT2_RANDOM_SPLIT_FP_FFNN_SEED_0_MODEL_PATH = os.path.join(DELQSAR_ROOT, 'experiments',
                                                  'models', 'triazine_SIRT2', 'FP-FFNN', 'random_seed_0.torch')

# SMARTS of substructure with highest weight from single substructure analysis
DD1S_CAIX_SEED_0_TOP_SUBSTRUCTURE_SMARTS = '[#7;D1;H2;R0;+0]-[#16;D4;H0;R0;+0](=[#8;D1;H0;R0;+0])(=[#8;D1;H0;R0;+0])-[#6;D3;H0;R1;+0]1:[#6;D2;H1;R1;+0]:[#6;D2;H1;R1;+0]:[#6;D3;H0;R1;+0]:[#6;D2;H1;R1;+0]:[#6;D2;H1;R1;+0]:1'
DD1S_CAIX_SEED_1_TOP_SUBSTRUCTURE_SMARTS = '[#7;D1;H2;R0;+0]-[#16;D4;H0;R0;+0](=[#8;D1;H0;R0;+0])(=[#8;D1;H0;R0;+0])-[#6;D3;H0;R1;+0]1:[#6;D2;H1;R1;+0]:[#6;D2;H1;R1;+0]:[#6;D3;H0;R1;+0]:[#6;D2;H1;R1;+0]:[#6;D2;H1;R1;+0]:1'
DD1S_CAIX_SEED_2_TOP_SUBSTRUCTURE_SMARTS = '[#7;D1;H2;R0;+0]-[#16;D4;H0;R0;+0](=[#8;D1;H0;R0;+0])(=[#8;D1;H0;R0;+0])-[#6;D3;H0;R1;+0]1:[#6;D2;H1;R1;+0]:[#6;D2;H1;R1;+0]:[#6;D3;H0;R1;+0]:[#6;D2;H1;R1;+0]:[#6;D2;H1;R1;+0]:1'
triazine_sEH_SEED_0_TOP_SUBSTRUCTURE_SMARTS = '[#6;D1;H3;R0;+0]-[#7;D3;H0;R0;+0](-[#6;D4;H0;R1;+0])-[#6;D3;H0;R1;+0]1:[#7;D2;H0;R1;+0]:[#6;D3;H0;R1;+0]:[#7;D2;H0;R1;+0]:[#6;D3;H0;R1;+0](-[#7;D2;H1;R0;+0]-[#6;D3;H1;R0;+0]):[#7;D2;H0;R1;+0]:1'
triazine_SIRT2_SEED_0_TOP_SUBSTRUCTURE_SMARTS = '[#6;D2;H0;R0;+0]-[#6;D3;H0;R1;+0]1:[#6;D2;H1;R1;+0]:[#16;D2;H0;R1;+0]:[#6;D3;H0;R1;+0](-[#6;D2;H2;R0;+0]-[#7;D2;H1;R0;+0]):[#7;D2;H0;R1;+0]:1'

# bit that the top substructure was mapped to (from single substructure analysis)
DD1S_CAIX_SEED_0_BASE_BIT = 997
DD1S_CAIX_SEED_1_BASE_BIT = 997
DD1S_CAIX_SEED_2_BASE_BIT = 997
triazine_sEH_SEED_0_BASE_BIT = 793
triazine_SIRT2_SEED_0_BASE_BIT = 348

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Arial') 
matplotlib.rc('text', usetex='false')

In [ ]:
def charge(a):
    """Returns a SMARTS substring describing the atomic charge."""
    if a.GetFormalCharge() >= 0:
        return f'+{a.GetFormalCharge()}'
    return f'-{abs(a.GetFormalCharge())}'
def getMorganFingerprintAtomSymbols(mol):
    """Generate custom atomSymbols based on the specificity of an atom
    definition used by Morgan Fingerprints. Namely,
        - atom ID
        - degree
        - number of Hs
        - ring membership
        - charge
    These are based on getConnectivityInvariants from FingerprintUtil.cpp at
    https://github.com/rdkit/rdkit/blob/75f03412ef151a4dc14dfee986e29c3690a4c071/Code/GraphMol/Fingerprints/FingerprintUtil.cpp#L254
    """
    atomSymbols = []
    for a in mol.GetAtoms():
        atomSymbols.append(
            f'[#{a.GetAtomicNum()};D{a.GetDegree()};H{a.GetTotalNumHs()};R{mol.GetRingInfo().NumAtomRings(a.GetIdx())};{charge(a)}]')
    return atomSymbols

In [ ]:
def GetWeightsForTwoBits(bit_id1, bit_id2): 
    """Returns 1) the differences in model predictions when the two specified bits are masked for each fingerprint,
    2) the average of these differences.
    Returns -inf for the average if the second specified bit is not set by any of the molecules.
    
    Parameters:
        - bit_id1: bit corresponding to top substructure
        - bit_id2: bit corresponding to other substructure
    """
    indices = np.squeeze(np.where(x[:,bit_id2]==1))
    x_hasbit2 = np.squeeze(x[indices, :])
    x_hasbits_masked = x_hasbit2.copy()
    if x_hasbit2.ndim == 1:
        num_mols = 1
        x_hasbits_masked[bit_id1] = 0
        x_hasbits_masked[bit_id2] = 0
    else:
        num_mols = x_hasbit2.shape[0]
        x_hasbits_masked[:, bit_id1] = 0
        x_hasbits_masked[:, bit_id2] = 0
    basePreds = basePreds_all[indices]
    newPreds = np.array(model.predict_on_x(np.array(x_hasbits_masked), device=DEVICE), dtype='float64')
    weights = np.subtract(basePreds, newPreds)
    if x_hasbit2.shape[0] != 0:
        avg_weight = np.sum(weights) / num_mols 
        return weights, avg_weight
    else: 
        print(f'bit {bit_id2} not set by any of the molecules')
        return weights, -np.inf

In [ ]:
def GetWeightsForSubstructure(weights, indices):
    """Returns 1) the molecule-level bit weights when the substructure of interest sets the bit,
    2) the average of these weights. 
    
    Parameters:
        - weights: molecule-level bit weights for a model and specific bit 
        - indices: indices in ::weights:: for the molecules with the substructure
    """
    weights_for_substructure = weights[indices]
    if isinstance(indices, list):
        avg_weight = np.sum(weights_for_substructure) / len(indices)
    else:
        avg_weight =  weights_for_substructure

    return weights_for_substructure, avg_weight

In [ ]:
def getFragmentForMolBit(smi, mol, mol_idx, atomSymbols, cpd_id, bit, info_all, submol_freq_distrib, 
                        smarts_to_smis, submol_to_cpd_indices, submol_to_bit, bits_to_draw):
    """Returns updated dictionaries and examples after searching for substructure(s) in the 
    specified molecule that set the specified bit.
    """
    molAdded = False
    examples = []
    for j, example in enumerate(info_all[mol_idx][bit]):
        atom = example[0]
        radius = example[1]
        env = Chem.FindAtomEnvironmentOfRadiusN(mol, radius, atom)
        atoms = set()
        for bidx in env:
            atoms.add(mol.GetBondWithIdx(bidx).GetBeginAtomIdx())
            atoms.add(mol.GetBondWithIdx(bidx).GetEndAtomIdx())
        if atoms:
            submol_smi = Chem.MolFragmentToSmiles(mol, atomsToUse=list(atoms), bondsToUse=env, 
                                            rootedAtAtom=atom, isomericSmiles=True, allBondsExplicit=True)
            submol_sm = Chem.MolFragmentToSmiles(mol, atomsToUse=list(atoms), atomSymbols=atomSymbols, 
                                            bondsToUse=env, isomericSmiles=True, allBondsExplicit=True)
            if submol_sm not in submol_freq_distrib[bit]:
                print(f'cpd_id: {cpd_id}')
                print(f'SMILES string: {smi}')
                print(f'bit ID: {bit}')
                print(f'(atom, radius): {(atom, radius)}')
                print(f'molecular fragment (SMILES): {submol_smi}')
                print(f'molecular fragment (SMARTS): {submol_sm}')
                print()
                submol_freq_distrib[bit][submol_sm] = 1 
                if submol_sm not in smarts_to_smis:
                    smarts_to_smis[submol_sm] = [submol_smi]
                else:
                    smarts_to_smis[submol_sm].append(submol_smi)   
                if submol_sm not in submol_to_bit:
                    submol_to_bit[submol_sm] = [bit]
                else:
                    submol_to_bit[submol_sm].append(bit)       
                examples.append(j)
                submol_to_cpd_indices[submol_sm] = [mol_idx]
                if not molAdded:
                    bits_to_draw.append((cpd_id, mol, bit, info_all[mol_idx]))
                    molAdded = True
            else:
                submol_freq_distrib[bit][submol_sm] += 1
                submol_to_cpd_indices[submol_sm].append(mol_idx)
        else:
            atom_smi = mol.GetAtomWithIdx(atom).GetSmarts()
            atom_sm = Chem.MolFragmentToSmiles(mol, 
                        atomsToUse=atom,
                        atomSymbols=atomSymbols,
                        isomericSmiles=True, 
                        allBondsExplicit=True)
            if atom_sm not in submol_freq_distrib[bit]:
                print(f'cpd_id: {cpd_id}')
                print(f'SMILES string: {smi}')
                print(f'bit ID: {bit}')
                print(f'(atom, radius): {(atom, radius)}')
                print(f'atom: {atom_sm}')
                print()
                submol_freq_distrib[bit][atom_sm] = 1
                if atom_sm not in smarts_to_smis:
                    smarts_to_smis[atom_sm] = [atom_smi]
                else:
                    smarts_to_smis[atom_sm].append(atom_smi)
                if atom_sm not in submol_to_bit:
                    submol_to_bit[atom_sm] = [bit]
                else:
                    submol_to_bit[atom_sm].append(bit)
                examples.append(j)
                submol_to_cpd_indices[atom_sm] = [mol_idx]
                if not molAdded:
                    bits_to_draw.append((cpd_id, mol, bit, info_all[mol_idx]))
                    molAdded = True
            else:
                submol_freq_distrib[bit][atom_sm] += 1
                submol_to_cpd_indices[atom_sm].append(mol_idx)
    return submol_freq_distrib, smarts_to_smis, submol_to_cpd_indices, submol_to_bit, bits_to_draw, examples

In [ ]:
DEVICE = None
if torch.cuda.is_available():
    DEVICE = 'cuda:0'

# DD1S CAIX (random split, seed 0)

## Load data

In [ ]:
os.environ["HDF5_USE_FILE_LOCKING"] = 'FALSE'
df_data_orig = pd.read_csv(os.path.join(DELQSAR_ROOT, 'experiments', 'datasets', 'DD1S_CAIX_QSAR.csv'))
hf = h5py.File(os.path.join(DELQSAR_ROOT, 'experiments', DD1S_FINGERPRINTS_FILENAME))
x = np.array(hf['all_fps'])
INPUT_SIZE = x.shape[1]   
hf.close()

In [ ]:
# reduce to molecules with top substructure
submol = DD1S_CAIX_SEED_0_TOP_SUBSTRUCTURE_SMARTS
base_bit = DD1S_CAIX_SEED_0_BASE_BIT
cpd_row_indices_for_bit = [idx for idx in np.squeeze(np.where(x[:,base_bit]==1))]
submol_to_cpd_indices = hkl.load('submol_to_cpd_indices_DD1S_CAIX_FP-FFNN_random_seed_0.hkl') # getting substructure SMARTS and cpd indices from single substructure analysis
x_withsubmol_rows = [cpd_row_indices_for_bit[i] for i in list(set(submol_to_cpd_indices[submol]))]

df_data = df_data_orig.iloc[x_withsubmol_rows]
x = x[x_withsubmol_rows, :]

## Load model

In [ ]:
model = models.MLP(INPUT_SIZE, [64, 64, 64],
            dropout=0.1)
model.load_state_dict(torch.load(DD1S_CAIX_RANDOM_SPLIT_FP_FFNN_SEED_0_MODEL_PATH))
if DEVICE:
    model = model.to(DEVICE)

## Bit analysis

### Calculate bit weights

In [ ]:
set_bit_ids = [bit_id for bit_id in tqdm(range(2048)) if any(x[:,bit_id]==1)]
print(f'Number of bits set by at least one molecule in the data set: {len(set_bit_ids)}')
if base_bit in set_bit_ids:
    set_bit_ids.remove(base_bit)

In [ ]:
basePreds_all = np.array(model.predict_on_x(x, device=DEVICE), dtype='float64')
if base_bit in set_bit_ids:
    set_bit_ids.remove(base_bit)
bit_to_weights = {bit_id: GetWeightsForTwoBits(base_bit, bit_id) for bit_id in tqdm(set_bit_ids)}
hkl.dump(bit_to_weights, 'twobits_bit_to_weights_DD1S_CAIX_FP-FFNN_random_seed_0.hkl', mode='w')

In [ ]:
# bit_to_weights_hkl = hkl.load('twobits_bit_to_weights_DD1S_CAIX_FP-FFNN_random_seed_0.hkl')
# bit_to_weights = {int(bit): weights for bit, weights in bit_to_weights_hkl.items()}

### Plot distribution of average bit weights

In [ ]:
avg_bit_weights = [bit_to_weights[b][1] for b in bit_to_weights.keys()]
print(f'Lowest average bit weight: {min(avg_bit_weights)}')
print(f'Highest average bit weight: {max(avg_bit_weights)}')

In [ ]:
# histogram of average bit weights (only including bits set by at least one molecule)
def make_hist_avg_bit_weights(zoomIn=False):
    fig = plt.figure(figsize=(3.5, 1.6), dpi=300)
    if zoomIn:
        bins = np.arange(0.064, 0.69, 0.005)
    else:
        bins = np.arange(0.064, 0.69, 0.01)
    _, bins, patches = plt.hist(
        np.clip(avg_bit_weights, 0.064, bins[-1]), 
        bins=bins,  
        density=False,
        zorder=2
    )
    fig.canvas.draw()
    ax = plt.gca()
    if zoomIn:
        ax.set_ylim([0, 8])
    ax.grid(zorder=1)
    ax.tick_params(labelsize=8)
    ax.set_xlabel('average weight', fontsize=8)
    ax.set_ylabel('number of bits', fontsize=8)
    plt.tight_layout()
    if zoomIn:
        plt.savefig(pathify(f'twobits_bit_weight_histogram_zoomed_in_DD1S_CAIX_FP-FFNN_random_seed_0.png'))
    else:
        plt.savefig(pathify(f'twobits_bit_weight_histogram_DD1S_CAIX_FP-FFNN_random_seed_0.png'))
    plt.show()

In [ ]:
make_hist_avg_bit_weights()

In [ ]:
make_hist_avg_bit_weights(zoomIn=True)

### Get bits of interest (based on average bit weight)

In [ ]:
bits_sorted_by_avg_weight = sorted(set_bit_ids, key = lambda b: bit_to_weights[b][1])
top_bits = bits_sorted_by_avg_weight[-5:]
top_bits.reverse()
bottom_bits = bits_sorted_by_avg_weight[:3]
bottom_bits.reverse()
print(f'Top bits: {top_bits}')
print(f'Bottom bits: {bottom_bits}')

In [ ]:
bits_of_interest = top_bits + bottom_bits
for b in bits_of_interest:
    print(f'Bit ID: {b}')
    print(f'Average weight: {bit_to_weights[b][1]}')
    mols_withbit = np.squeeze(np.where(x[:,b]==1))
    if mols_withbit.ndim > 0:
        print(f'Number of molecules with the bit: {len(mols_withbit)}')
    else:
        print('Number of molecules with the bit: 1')
    print()

In [ ]:
bit_to_cpd_row_indices = {bit: list([idx for idx in np.squeeze(np.where(x[:,bit]==1))]) for bit in bits_of_interest}
for item in bit_to_cpd_row_indices.items():
    print(f'Bit ID: {item[0]}')
    print(f'Number of molecules with the bit: {len(item[1])}')
    print()

### Plot distributions of molecule-level bit weights

In [ ]:
# histogram of molecule-level bit weights
def make_hist_mol_level_bit_weights(bit_id, x_lb, x_ub, stepsize_noZoom, stepsize_zoom, zoomIn=False,
                                    zoomIn_y_ub=None, xticks=None, yticks=None):
    weights = bit_to_weights[bit_id][0]
    fig = plt.figure(figsize=(3.5, 1.6), dpi=300)
    if zoomIn:
        bins = np.arange(x_lb, x_ub, stepsize_zoom)
    else:
        bins = np.arange(x_lb, x_ub, stepsize_noZoom) 
    _, bins, patches = plt.hist(
        np.clip(weights, x_lb, bins[-1]), 
        bins=bins,  
        density=False,
        zorder=2
    )
    fig.canvas.draw()
    ax = plt.gca()
    if zoomIn:
        ax.set_ylim([0, zoomIn_y_ub])
    ax.grid(zorder=1)
    ax.tick_params(labelsize=8)
    if xticks:
        plt.xticks(xticks)
    if yticks:
        plt.yticks(yticks)
    ax.set_xlabel('weight', fontsize=8)
    ax.set_ylabel('number of molecules', fontsize=8)
    ax.set_title(f'bit {bit_id}', fontsize=8)
    plt.tight_layout()
    if zoomIn:
        plt.savefig(pathify(f'twobits_mol-level_bit_weight_histogram_bit_{bit_id}_zoomed_in_DD1S_CAIX_FP-FFNN_random_seed_0.png'))
    else:
        plt.savefig(pathify(f'twobits_mol-level_bit_weight_histogram_bit_{bit_id}_DD1S_CAIX_FP-FFNN_random_seed_0.png'))
    plt.show()

In [ ]:
bit = 1262
print(f'Bit {bit}')
print(f'Lowest molecule-level bit weight: {min(bit_to_weights[bit][0])}')
print(f'Highest molecule-level bit weight: {max(bit_to_weights[bit][0])}')

make_hist_mol_level_bit_weights(bit, 0.672, 0.697, 0.0005, None, yticks=[0, 1])

In [ ]:
bit = 1802
print(f'Bit {bit}')
print(f'Lowest molecule-level bit weight: {min(bit_to_weights[bit][0])}')
print(f'Highest molecule-level bit weight: {max(bit_to_weights[bit][0])}')

make_hist_mol_level_bit_weights(bit, 0.620, 0.641, 0.0004, None)

In [ ]:
bit = 1953
print(f'Bit {bit}')
print(f'Lowest molecule-level bit weight: {min(bit_to_weights[bit][0])}')
print(f'Highest molecule-level bit weight: {max(bit_to_weights[bit][0])}')

make_hist_mol_level_bit_weights(bit, 0.589, 0.640, 0.000975, None)

In [ ]:
bit = 833
print(f'Bit {bit}')
print(f'Lowest molecule-level bit weight: {min(bit_to_weights[bit][0])}')
print(f'Highest molecule-level bit weight: {max(bit_to_weights[bit][0])}')

make_hist_mol_level_bit_weights(bit, 0.139, 1.285, 0.02, None)

In [ ]:
bit = 1489
print(f'Bit {bit}')
print(f'Lowest molecule-level bit weight: {min(bit_to_weights[bit][0])}')
print(f'Highest molecule-level bit weight: {max(bit_to_weights[bit][0])}')

make_hist_mol_level_bit_weights(bit, 0.132, 1.258, 0.02, None)

In [ ]:
bit = 1736
print(f'Bit {bit}')
print(f'Lowest molecule-level bit weight: {min(bit_to_weights[bit][0])}')
print(f'Highest molecule-level bit weight: {max(bit_to_weights[bit][0])}')

make_hist_mol_level_bit_weights(bit, 0.00954, 0.148, 0.0028, None)

In [ ]:
bit = 1844
print(f'Bit {bit}')
print(f'Lowest molecule-level bit weight: {min(bit_to_weights[bit][0])}')
print(f'Highest molecule-level bit weight: {max(bit_to_weights[bit][0])}')

make_hist_mol_level_bit_weights(bit, 0.0365, 0.0953, 0.00115, None, yticks=[0,1])

In [ ]:
bit = 1135
print(f'Bit {bit}')
print(f'Lowest molecule-level bit weight: {min(bit_to_weights[bit][0])}')
print(f'Highest molecule-level bit weight: {max(bit_to_weights[bit][0])}')

make_hist_mol_level_bit_weights(bit, 0.0386, 0.0898, 0.001, None)

### Get and visualize substructures

In [ ]:
bits_to_draw = []
submol_freq_distrib = {bit: {} for bit in bits_of_interest} # store frequency distribution of substructures that 
                                                            # set each bit
smarts_to_smis = {}
submol_to_cpd_indices = {} # mapping to indices in df_data_hasbit
submol_to_bit = {}
examples_all = {bit: {} for bit in bits_of_interest} # check if there's more than one distinct bit-setting 
                                                     # substructure in the same molecule

for bit in tqdm(bits_of_interest):
    df_data_hasbit = df_data.iloc[bit_to_cpd_row_indices[bit]]
    smis = df_data_hasbit['smiles']
    featurizer = featurizers.FingerprintFeaturizer()
    _, info_all = featurizer.prepare_x(df_data_hasbit, bitInfo=True)
    for i, smi in enumerate(smis):
        mol = Chem.MolFromSmiles(smi)
        atomSymbols = getMorganFingerprintAtomSymbols(mol)
        cpd_id = int(df_data[df_data['smiles']==smi]['cpd_id'].to_numpy()[0])
        if bit not in info_all[i]:
            continue
        fragment_logs = getFragmentForMolBit(smi, mol, i, atomSymbols, cpd_id, bit, info_all, submol_freq_distrib, 
                        smarts_to_smis, submol_to_cpd_indices, submol_to_bit, bits_to_draw)
        submol_freq_distrib = fragment_logs[0]
        smarts_to_smis = fragment_logs[1]
        submol_to_cpd_indices = fragment_logs[2]
        submol_to_bit = fragment_logs[3]
        bits_to_draw = fragment_logs[4]
        examples = fragment_logs[5]
        if examples:
            examples_all[bit][i] = examples

In [ ]:
hkl.dump(submol_freq_distrib, 'twobits_submol_freq_distrib_DD1S_CAIX_FP-FFNN_random_seed_0.hkl', mode='w')
hkl.dump(smarts_to_smis, 'twobits_smarts_to_smis_DD1S_CAIX_FP-FFNN_random_seed_0.hkl', mode='w')
hkl.dump(submol_to_cpd_indices, 'twobits_submol_to_cpd_indices_DD1S_CAIX_FP-FFNN_random_seed_0.hkl', mode='w')
hkl.dump(submol_to_bit, 'twobits_submol_to_bit_DD1S_CAIX_FP-FFNN_random_seed_0.hkl', mode='w')
hkl.dump(bits_to_draw, 'twobits_bits_to_draw_DD1S_CAIX_FP-FFNN_random_seed_0.hkl', mode='w')
hkl.dump(examples_all, 'twobits_examples_all_DD1S_CAIX_FP-FFNN_random_seed_0.hkl', mode='w')

In [ ]:
# submol_freq_distrib_hkl = hkl.load('twobits_submol_freq_distrib_DD1S_CAIX_FP-FFNN_random_seed_0.hkl')
# submol_freq_distrib = {int(bit): d for bit, d in submol_freq_distrib_hkl.items()}
# smarts_to_smis = hkl.load('twobits_smarts_to_smis_DD1S_CAIX_FP-FFNN_random_seed_0.hkl')
# submol_to_cpd_indices = hkl.load('twobits_submol_to_cpd_indices_DD1S_CAIX_FP-FFNN_random_seed_0.hkl')
# submol_to_bit = hkl.load('twobits_submol_to_bit_DD1S_CAIX_FP-FFNN_random_seed_0.hkl')
# bits_to_draw_hkl = hkl.load('twobits_bits_to_draw_DD1S_CAIX_FP-FFNN_random_seed_0.hkl')
# bits_to_draw = [(int(item[0]), Chem.MolFromSmiles(df_data_orig.iloc[int(item[0])-1]['smiles']), int(item[2]), 
#         {int(bit): tuple([(int(an[0]), int(an[1])) for an in ans]) for bit, ans in item[3].items()}) for item in bits_to_draw_hkl]
# examples_all_hkl = hkl.load('twobits_examples_all_DD1S_CAIX_FP-FFNN_random_seed_0.hkl')
# examples_all = {int(bit): {int(mol_idx): [int(ex_num) for ex_num in ex_nums] for mol_idx, ex_nums in exs.items()} for bit, exs in examples_all_hkl.items()}

In [ ]:
examples_all

In [ ]:
for item in smarts_to_smis.items():
    print(f'SMARTS: {item[0]}')
    print(f'SMILES: {np.squeeze(item[1])}')
    print()
print()
print(f'Number of distinct SMARTS: {len(set(smarts_to_smis.keys()))}')
print(f'Number of distinct SMILES: {len(set([j for i in smarts_to_smis.values() for j in i]))}')

In [ ]:
for bit in submol_freq_distrib:
    print(f'Bit ID: {bit}')
    for submol in submol_freq_distrib[bit]:
        print(f'Substructure (SMARTS): {submol}')
        print(f'Frequency: {submol_freq_distrib[bit][submol]}')
    print()

In [ ]:
_bits_to_draw = [bit[1:] for bit in bits_to_draw]
d = Draw.DrawMorganBits(_bits_to_draw, molsPerRow=4, aromaticColor=None, ringColor=None, 
                        legends=[f'cpd_id {bit[0]}, bit: {bit[2]}' for bit in bits_to_draw], subImgSize=(600, 600))
d.save(pathify(f'twobits_visualization_DD1S_CAIX_FP-FFNN_random_seed_0.png'))
d

## Substructure analysis

### Number substructures

In [ ]:
# numbering correponds to ordering of bars from left to right on plot of substructure weights (see below)
ctr = 1
submol_to_id = {}
for bit in bits_of_interest:
    submols = sorted(submol_freq_distrib[bit].keys(), key = lambda x: submol_freq_distrib[bit][x], reverse=True)
    for s in submols:
        submol_to_id[s] = ctr
        ctr += 1

In [ ]:
hkl.dump(submol_to_id, 'twobits_submol_to_id_DD1S_CAIX_FP-FFNN_random_seed_0.hkl', mode='w')

In [ ]:
# submol_to_id = hkl.load('twobits_submol_to_id_DD1S_CAIX_FP-FFNN_random_seed_0.hkl')

In [ ]:
for item in sorted(submol_to_id.items(), key=lambda x: x[1]):
    print(item[1])
    print(item[0])
    print(np.squeeze(smarts_to_smis[item[0]]))
    print()

### Calculate substructure weights

In [ ]:
substruct_to_weight = {}
for submol in submol_to_cpd_indices:
    bit = int(np.squeeze(submol_to_bit[submol]))
    substruct_to_weight[submol] = GetWeightsForSubstructure(bit_to_weights[bit][0], list(set(submol_to_cpd_indices[submol])))[1]

In [ ]:
for item in sorted(substruct_to_weight.items(), key=lambda x: submol_to_id[x[0]]):
    print(f'Substructure (SMARTS): {item[0]}')
    print()
    print(f'Weight: {item[1]}')
    print()
    print()

### Plot substructure weights

In [ ]:
substructs = sorted(substruct_to_weight.keys(), key=lambda s: submol_to_id[s])
weights = [substruct_to_weight[s] for s in substructs]
bars = []
for bit in bits_of_interest:
    _bars = []
    substruct_smarts_for_bit = submol_freq_distrib[bit].keys()
    for s in substructs:
        if s in substruct_smarts_for_bit:
            _bars.append(substruct_to_weight[s])
    bars.append(_bars)

barWidth = 2
offset = 2.2
space = 4

pos = len(bars[0])*barWidth / 2

In [ ]:
r1 = [pos - 0.5*barWidth - 0.5*(offset-barWidth), pos + 0.5*barWidth + 0.5*(offset-barWidth)]
pos += space + 1.5*barWidth + (offset-barWidth)
r2 = [pos]
pos += space + 1.5*barWidth + (offset-barWidth)
r3 = [pos]
pos += space + 1.5*barWidth + (offset-barWidth)
r4 = [pos]
pos += space + 1.5*barWidth + (offset-barWidth)
r5 = [pos - 0.5*barWidth - 0.5*(offset-barWidth), pos + 0.5*barWidth + 0.5*(offset-barWidth)]
pos += space + 1.5*barWidth + (offset-barWidth)
r6 = [pos]
pos += space + 1.5*barWidth + (offset-barWidth)
r7 = [pos]
pos += space + 1.5*barWidth + (offset-barWidth)
r8 = [pos - 0.5*barWidth - 0.5*(offset-barWidth), pos + 0.5*barWidth + 0.5*(offset-barWidth)]
rs = [r1, r2, r3, r4, r5, r6, r7, r8]

In [ ]:
fig = plt.figure(figsize=(7, 4), dpi=300)

rects = []
for bit in range(len(bars)):
    for i in range(len(rs[bit])):
        rects.append(plt.bar(rs[bit][i], bars[bit][i], color='#1f77b4', width=barWidth, zorder=2)[0])

for rect in rects:
    height = rect.get_height()
    plt.annotate('{:.3f}'.format(height),
                xy=(rect.get_x() + rect.get_width() / 2, height),
                xytext=(0, 3),  
                textcoords="offset points",
                ha='center', va='bottom',
                fontsize=4.5)

fig.canvas.draw()
ax = plt.gca()
ax.grid(zorder=1)
ax.set_xlabel('bit ID', fontsize=9)
ax.set_ylabel('substructure weight', fontsize=9)
ax.tick_params(labelsize=9)
ax.set_xticks([((len(bars[0])*barWidth / 2) + i*(space + 1.5*barWidth + (offset-barWidth))) for i in range(len(bars))])
ax.set_xticklabels([str(bit_id) for bit_id in bits_of_interest], ha='center')
ax.tick_params(axis='x', length=0)
plt.tight_layout()
plt.savefig(pathify(f'twobits_substructure_weights_DD1S_CAIX_FP-FFNN_random_seed_0.png'))
plt.show()

# DD1S CAIX (random split, seed 1)

## Load data

In [ ]:
os.environ["HDF5_USE_FILE_LOCKING"] = 'FALSE'
df_data_orig = pd.read_csv(os.path.join(DELQSAR_ROOT, 'experiments', 'datasets', 'DD1S_CAIX_QSAR.csv'))
hf = h5py.File(os.path.join(DELQSAR_ROOT, 'experiments', DD1S_FINGERPRINTS_FILENAME))
x = np.array(hf['all_fps'])
INPUT_SIZE = x.shape[1]   
hf.close()

In [ ]:
# reduce to molecules with top substructure
submol = DD1S_CAIX_SEED_1_TOP_SUBSTRUCTURE_SMARTS
base_bit = DD1S_CAIX_SEED_1_BASE_BIT
cpd_row_indices_for_bit = [idx for idx in np.squeeze(np.where(x[:,base_bit]==1))]
submol_to_cpd_indices = hkl.load('submol_to_cpd_indices_DD1S_CAIX_FP-FFNN_random_seed_1.hkl') # getting substructure SMARTS and cpd indices from single bit substructure analysis
x_withsubmol_rows = [cpd_row_indices_for_bit[i] for i in list(set(submol_to_cpd_indices[submol]))]

df_data = df_data_orig.iloc[x_withsubmol_rows]
x = x[x_withsubmol_rows, :]

## Load model

In [ ]:
input_size = x.shape[1]     
model = models.MLP(INPUT_SIZE, [128, 64, 32],
            dropout=0.4)
model.load_state_dict(torch.load(DD1S_CAIX_RANDOM_SPLIT_FP_FFNN_SEED_1_MODEL_PATH))
if DEVICE:
    model = model.to(DEVICE)

## Bit analysis

### Calculate bit weights

In [ ]:
set_bit_ids = [bit_id for bit_id in tqdm(range(2048)) if any(x[:,bit_id]==1)]
print(f'Number of bits set by at least one molecule in the data set: {len(set_bit_ids)}')

In [ ]:
basePreds_all = np.array(model.predict_on_x(x, device=DEVICE), dtype='float64')
if base_bit in set_bit_ids:
    set_bit_ids.remove(base_bit)
bit_to_weights = {bit_id: GetWeightsForTwoBits(base_bit, bit_id) for bit_id in tqdm(set_bit_ids)}
hkl.dump(bit_to_weights, 'twobits_bit_to_weights_DD1S_CAIX_FP-FFNN_random_seed_1.hkl', mode='w')

In [ ]:
# bit_to_weights_hkl = hkl.load('twobits_bit_to_weights_DD1S_CAIX_FP-FFNN_random_seed_1.hkl')
# bit_to_weights = {int(bit): weights for bit, weights in bit_to_weights_hkl.items()}

### Plot distribution of average bit weights

In [ ]:
avg_bit_weights = [bit_to_weights[b][1] for b in bit_to_weights.keys()]
print(f'Lowest average bit weight: {min(avg_bit_weights)}')
print(f'Highest average bit weight: {max(avg_bit_weights)}')

In [ ]:
# histogram of average bit weights (only including bits set by at least one molecule)
def make_hist_avg_bit_weights(zoomIn=False):
    fig = plt.figure(figsize=(3.5, 1.6), dpi=300)
    if zoomIn:
        bins = np.arange(0.054, 0.49, 0.0045)
    else:
        bins = np.arange(0.054, 0.49, 0.006)
    _, bins, patches = plt.hist(
        np.clip(avg_bit_weights, 0.054, bins[-1]), 
        bins=bins,  
        density=False,
        zorder=2
    )
    fig.canvas.draw()
    ax = plt.gca()
    if zoomIn:
        ax.set_ylim([0, 8])
    ax.grid(zorder=1)
    ax.tick_params(labelsize=8)
    ax.set_xlabel('average weight', fontsize=8)
    ax.set_ylabel('number of bits', fontsize=8)
    plt.tight_layout()
    if zoomIn:
        plt.savefig(pathify(f'twobits_bit_weight_histogram_zoomed_in_DD1S_CAIX_FP-FFNN_random_seed_1.png'))
    else:
        plt.savefig(pathify(f'twobits_bit_weight_histogram_DD1S_CAIX_FP-FFNN_random_seed_1.png'))
    plt.show()

In [ ]:
make_hist_avg_bit_weights()

In [ ]:
make_hist_avg_bit_weights(zoomIn=True)

### Get bits of interest (based on average bit weight)

In [ ]:
bits_sorted_by_avg_weight = sorted(set_bit_ids, key = lambda b: bit_to_weights[b][1])
top_bits = bits_sorted_by_avg_weight[-5:]
top_bits.reverse()
bottom_bits = bits_sorted_by_avg_weight[:3]
bottom_bits.reverse()
print(f'Top bits: {top_bits}')
print(f'Bottom bits: {bottom_bits}')

In [ ]:
bits_of_interest = top_bits + bottom_bits
for b in bits_of_interest:
    print(f'Bit ID: {b}')
    print(f'Average weight: {bit_to_weights[b][1]}')
    mols_withbit = np.squeeze(np.where(x[:,b]==1))
    if mols_withbit.ndim > 0:
        print(f'Number of molecules with the bit: {len(mols_withbit)}')
    else:
        print('Number of molecules with the bit: 1')
    print()

In [ ]:
bit_to_cpd_row_indices = {bit: list([idx for idx in np.squeeze(np.where(x[:,bit]==1))]) for bit in bits_of_interest}
for item in bit_to_cpd_row_indices.items():
    print(f'Bit ID: {item[0]}')
    print(f'Number of molecules with the bit: {len(item[1])}')
    print()

### Plot distributions of molecule-level bit weights

In [ ]:
# histogram of molecule-level bit weights
def make_hist_mol_level_bit_weights(bit_id, x_lb, x_ub, stepsize_noZoom, stepsize_zoom, zoomIn=False,
                                    zoomIn_y_ub=None, xticks=None, yticks=None):
    weights = bit_to_weights[bit_id][0]
    fig = plt.figure(figsize=(3.5, 1.6), dpi=300)
    if zoomIn:
        bins = np.arange(x_lb, x_ub, stepsize_zoom)
    else:
        bins = np.arange(x_lb, x_ub, stepsize_noZoom) 
    _, bins, patches = plt.hist(
        np.clip(weights, x_lb, bins[-1]), 
        bins=bins,  
        density=False,
        zorder=2
    )
    fig.canvas.draw()
    ax = plt.gca()
    if zoomIn:
        ax.set_ylim([0, zoomIn_y_ub])
    ax.grid(zorder=1)
    ax.tick_params(labelsize=8)
    if xticks:
        plt.xticks(xticks)
    if yticks:
        plt.yticks(yticks)
    ax.set_xlabel('weight', fontsize=8)
    ax.set_ylabel('number of molecules', fontsize=8)
    ax.set_title(f'bit {bit_id}', fontsize=8)
    plt.tight_layout()
    if zoomIn:
        plt.savefig(pathify(f'twobits_mol-level_bit_weight_histogram_bit_{bit_id}_zoomed_in_DD1S_CAIX_FP-FFNN_random_seed_1.png'))
    else:
        plt.savefig(pathify(f'twobits_mol-level_bit_weight_histogram_bit_{bit_id}_DD1S_CAIX_FP-FFNN_random_seed_1.png'))
    plt.show()

In [ ]:
bit = 833
print(f'Bit {bit}')
print(f'Lowest molecule-level bit weight: {min(bit_to_weights[bit][0])}')
print(f'Highest molecule-level bit weight: {max(bit_to_weights[bit][0])}')

make_hist_mol_level_bit_weights(bit, 0.311, 0.524, 0.004, None)

In [ ]:
bit = 1785
print(f'Bit {bit}')
print(f'Lowest molecule-level bit weight: {min(bit_to_weights[bit][0])}')
print(f'Highest molecule-level bit weight: {max(bit_to_weights[bit][0])}')

make_hist_mol_level_bit_weights(bit, 0.305, 0.520, 0.0045, None)

In [ ]:
bit = 1489
print(f'Bit {bit}')
print(f'Lowest molecule-level bit weight: {min(bit_to_weights[bit][0])}')
print(f'Highest molecule-level bit weight: {max(bit_to_weights[bit][0])}')

make_hist_mol_level_bit_weights(bit, 0.287, 0.506, 0.004, None)

In [ ]:
bit = 1197
print(f'Bit {bit}')
print(f'Lowest molecule-level bit weight: {min(bit_to_weights[bit][0])}')
print(f'Highest molecule-level bit weight: {max(bit_to_weights[bit][0])}')

make_hist_mol_level_bit_weights(bit, 0.290, 0.499, 0.004, None)

In [ ]:
bit = 168
print(f'Bit {bit}')
print(f'Lowest molecule-level bit weight: {min(bit_to_weights[bit][0])}')
print(f'Highest molecule-level bit weight: {max(bit_to_weights[bit][0])}')

make_hist_mol_level_bit_weights(bit, 0.278, 0.485, 0.004, None)

In [ ]:
bit = 1705
print(f'Bit {bit}')
print(f'Lowest molecule-level bit weight: {min(bit_to_weights[bit][0])}')
print(f'Highest molecule-level bit weight: {max(bit_to_weights[bit][0])}')

make_hist_mol_level_bit_weights(bit, 0.0713, 0.127, 0.00089, None)

In [ ]:
bit = 1537
print(f'Bit {bit}')
print(f'Lowest molecule-level bit weight: {min(bit_to_weights[bit][0])}')
print(f'Highest molecule-level bit weight: {max(bit_to_weights[bit][0])}')

make_hist_mol_level_bit_weights(bit, 0.0452, 0.0910, 0.0008, None)

In [ ]:
bit = 1736
print(f'Bit {bit}')
print(f'Lowest molecule-level bit weight: {min(bit_to_weights[bit][0])}')
print(f'Highest molecule-level bit weight: {max(bit_to_weights[bit][0])}')

make_hist_mol_level_bit_weights(bit, 0.0314, 0.0747, 0.0007, None, yticks=[0,1])

### Get and visualize substructures

In [ ]:
bits_to_draw = []
submol_freq_distrib = {bit: {} for bit in bits_of_interest} # store frequency distribution of substructures that 
                                                            # set each bit
smarts_to_smis = {}
submol_to_cpd_indices = {} # mapping to indices in df_data_hasbit
submol_to_bit = {}
examples_all = {bit: {} for bit in bits_of_interest} # check if there's more than one distinct bit-setting 
                                                     # substructure in the same molecule

for bit in tqdm(bits_of_interest):
    df_data_hasbit = df_data.iloc[bit_to_cpd_row_indices[bit]]
    smis = df_data_hasbit['smiles']
    featurizer = featurizers.FingerprintFeaturizer()
    _, info_all = featurizer.prepare_x(df_data_hasbit, bitInfo=True)
    for i, smi in enumerate(smis):
        mol = Chem.MolFromSmiles(smi)
        atomSymbols = getMorganFingerprintAtomSymbols(mol)
        cpd_id = int(df_data[df_data['smiles']==smi]['cpd_id'].to_numpy()[0])
        if bit not in info_all[i]:
            continue
        fragment_logs = getFragmentForMolBit(smi, mol, i, atomSymbols, cpd_id, bit, info_all, submol_freq_distrib, 
                        smarts_to_smis, submol_to_cpd_indices, submol_to_bit, bits_to_draw)
        submol_freq_distrib = fragment_logs[0]
        smarts_to_smis = fragment_logs[1]
        submol_to_cpd_indices = fragment_logs[2]
        submol_to_bit = fragment_logs[3]
        bits_to_draw = fragment_logs[4]
        examples = fragment_logs[5]
        if examples:
            examples_all[bit][i] = examples

In [ ]:
hkl.dump(submol_freq_distrib, 'twobits_submol_freq_distrib_DD1S_CAIX_FP-FFNN_random_seed_1.hkl', mode='w')
hkl.dump(smarts_to_smis, 'twobits_smarts_to_smis_DD1S_CAIX_FP-FFNN_random_seed_1.hkl', mode='w')
hkl.dump(submol_to_cpd_indices, 'twobits_submol_to_cpd_indices_DD1S_CAIX_FP-FFNN_random_seed_1.hkl', mode='w')
hkl.dump(submol_to_bit, 'twobits_submol_to_bit_DD1S_CAIX_FP-FFNN_random_seed_1.hkl', mode='w')
hkl.dump(bits_to_draw, 'twobits_bits_to_draw_DD1S_CAIX_FP-FFNN_random_seed_1.hkl', mode='w')
hkl.dump(examples_all, 'twobits_examples_all_DD1S_CAIX_FP-FFNN_random_seed_1.hkl', mode='w')

In [ ]:
# submol_freq_distrib_hkl = hkl.load('twobits_submol_freq_distrib_DD1S_CAIX_FP-FFNN_random_seed_1.hkl')
# submol_freq_distrib = {int(bit): d for bit, d in submol_freq_distrib_hkl.items()}
# smarts_to_smis = hkl.load('twobits_smarts_to_smis_DD1S_CAIX_FP-FFNN_random_seed_1.hkl')
# submol_to_cpd_indices = hkl.load('twobits_submol_to_cpd_indices_DD1S_CAIX_FP-FFNN_random_seed_1.hkl')
# submol_to_bit = hkl.load('twobits_submol_to_bit_DD1S_CAIX_FP-FFNN_random_seed_1.hkl')
# bits_to_draw_hkl = hkl.load('twobits_bits_to_draw_DD1S_CAIX_FP-FFNN_random_seed_1.hkl')
# bits_to_draw = [(int(item[0]), Chem.MolFromSmiles(df_data_orig.iloc[int(item[0])-1]['smiles']), int(item[2]), 
#         {int(bit): tuple([(int(an[0]), int(an[1])) for an in ans]) for bit, ans in item[3].items()}) for item in bits_to_draw_hkl]
# examples_all_hkl = hkl.load('twobits_examples_all_DD1S_CAIX_FP-FFNN_random_seed_1.hkl')
# examples_all = {int(bit): {int(mol_idx): [int(ex_num) for ex_num in ex_nums] for mol_idx, ex_nums in exs.items()} for bit, exs in examples_all_hkl.items()}

In [ ]:
examples_all

In [ ]:
for item in smarts_to_smis.items():
    print(f'SMARTS: {item[0]}')
    print(f'SMILES: {np.squeeze(item[1])}')
    print()
print()
print(f'Number of distinct SMARTS: {len(set(smarts_to_smis.keys()))}')
print(f'Number of distinct SMILES: {len(set([j for i in smarts_to_smis.values() for j in i]))}')

In [ ]:
for bit in submol_freq_distrib:
    print(f'Bit ID: {bit}')
    for submol in submol_freq_distrib[bit]:
        print(f'Substructure (SMARTS): {submol}')
        print(f'Frequency: {submol_freq_distrib[bit][submol]}')
    print()

In [ ]:
_bits_to_draw = [bit[1:] for bit in bits_to_draw]
d = Draw.DrawMorganBits(_bits_to_draw, molsPerRow=4, aromaticColor=None, ringColor=None, 
                        legends=[f'cpd_id {bit[0]}, bit: {bit[2]}' for bit in bits_to_draw], subImgSize=(600, 600))
d.save(pathify(f'twobits_visualization_DD1S_CAIX_FP-FFNN_random_seed_1.png'))
d

## Substructure analysis

### Number substructures

In [ ]:
# numbering correponds to ordering of bars from left to right on plot of substructure weights (see below)
ctr = 1
submol_to_id = {}
for bit in bits_of_interest:
    submols = sorted(submol_freq_distrib[bit].keys(), key = lambda x: submol_freq_distrib[bit][x], reverse=True)
    for s in submols:
        submol_to_id[s] = ctr
        ctr += 1

In [ ]:
hkl.dump(submol_to_id, 'twobits_submol_to_id_DD1S_CAIX_FP-FFNN_random_seed_1.hkl', mode='w')

In [ ]:
# submol_to_id = hkl.load('twobits_submol_to_id_DD1S_CAIX_FP-FFNN_random_seed_1.hkl')

In [ ]:
for item in sorted(submol_to_id.items(), key=lambda x: x[1]):
    print(item[1])
    print(item[0])
    print(np.squeeze(smarts_to_smis[item[0]]))
    print()

### Calculate substructure weights

In [ ]:
substruct_to_weight = {}
for submol in submol_to_cpd_indices:
    bit = int(np.squeeze(submol_to_bit[submol]))
    substruct_to_weight[submol] = GetWeightsForSubstructure(bit_to_weights[bit][0], list(set(submol_to_cpd_indices[submol])))[1]

In [ ]:
for item in sorted(substruct_to_weight.items(), key=lambda x: submol_to_id[x[0]]):
    print(f'Substructure (SMARTS): {item[0]}')
    print()
    print(f'Weight: {item[1]}')
    print()
    print()

### Plot substructure weights

In [ ]:
substructs = sorted(substruct_to_weight.keys(), key=lambda s: submol_to_id[s])
weights = [substruct_to_weight[s] for s in substructs]
bars = []
for bit in bits_of_interest:
    _bars = []
    substruct_smarts_for_bit = submol_freq_distrib[bit].keys()
    for s in substructs:
        if s in substruct_smarts_for_bit:
            _bars.append(substruct_to_weight[s])
    bars.append(_bars)

barWidth = 2
offset = 2.2
space = 4

pos = len(bars[0])*barWidth / 2

In [ ]:
r1 = [pos]
pos += space + 1.5*barWidth + (offset-barWidth)
r2 = [pos - 0.5*barWidth - 0.5*(offset-barWidth), pos + 0.5*barWidth + 0.5*(offset-barWidth)]
pos += space + 1.5*barWidth + (offset-barWidth)
r3 = [pos - 0.5*barWidth - 0.5*(offset-barWidth), pos + 0.5*barWidth + 0.5*(offset-barWidth)]
pos += space + 1.5*barWidth + (offset-barWidth)
r4 = [pos - 0.5*barWidth - 0.5*(offset-barWidth), pos + 0.5*barWidth + 0.5*(offset-barWidth)]
pos += space + 1.5*barWidth + (offset-barWidth)
r5 = [pos - barWidth - (offset-barWidth), pos, pos + barWidth + (offset-barWidth)]
pos += space + 1.5*barWidth + (offset-barWidth)
r6 = [pos - 0.5*barWidth - 0.5*(offset-barWidth), pos + 0.5*barWidth + 0.5*(offset-barWidth)]
pos += space + 1.5*barWidth + (offset-barWidth)
r7 = [pos - barWidth - (offset-barWidth), pos, pos + barWidth + (offset-barWidth)]
pos += space + 1.5*barWidth + (offset-barWidth)
r8 = [pos]
rs = [r1, r2, r3, r4, r5, r6, r7, r8]

In [ ]:
fig = plt.figure(figsize=(7, 4), dpi=300)

rects = []
for bit in range(len(bars)):
    for i in range(len(rs[bit])):
        rects.append(plt.bar(rs[bit][i], bars[bit][i], color='#1f77b4', width=barWidth, zorder=2)[0])
        
for rect in rects:
    height = rect.get_height()
    plt.annotate('{:.3f}'.format(height),
                xy=(rect.get_x() + rect.get_width() / 2, height),
                xytext=(0, 3),  
                textcoords="offset points",
                ha='center', va='bottom',
                fontsize=4.5)
        
fig.canvas.draw()
ax = plt.gca()
ax.grid(zorder=1)
ax.set_xlabel('bit ID', fontsize=9)
ax.set_ylabel('substructure weight', fontsize=9)
ax.tick_params(labelsize=9)
ax.set_xticks([((len(bars[0])*barWidth / 2) + i*(space + 1.5*barWidth + (offset-barWidth))) for i in range(len(bars))])
ax.set_xticklabels([str(bit_id) for bit_id in bits_of_interest], ha='center')
ax.tick_params(axis='x', length=0)
plt.tight_layout()
plt.savefig(pathify(f'twobits_substructure_weights_DD1S_CAIX_FP-FFNN_random_seed_1.png'))
plt.show()

# DD1S CAIX (random split, seed 2)

## Load data

In [ ]:
os.environ["HDF5_USE_FILE_LOCKING"] = 'FALSE'
df_data_orig = pd.read_csv(os.path.join(DELQSAR_ROOT, 'experiments', 'datasets', 'DD1S_CAIX_QSAR.csv'))
hf = h5py.File(os.path.join(DELQSAR_ROOT, 'experiments', DD1S_FINGERPRINTS_FILENAME))
x = np.array(hf['all_fps'])
INPUT_SIZE = x.shape[1]   
hf.close()

In [ ]:
# reduce to molecules with top substructure
submol = DD1S_CAIX_SEED_2_TOP_SUBSTRUCTURE_SMARTS
base_bit = DD1S_CAIX_SEED_2_BASE_BIT
cpd_row_indices_for_bit = [idx for idx in np.squeeze(np.where(x[:,base_bit]==1))]
submol_to_cpd_indices = hkl.load('submol_to_cpd_indices_DD1S_CAIX_FP-FFNN_random_seed_2.hkl') # getting substructure SMARTS and cpd indices from single bit substructure analysis
x_withsubmol_rows = [cpd_row_indices_for_bit[i] for i in list(set(submol_to_cpd_indices[submol]))]

df_data = df_data_orig.iloc[x_withsubmol_rows]
x = x[x_withsubmol_rows, :]

## Load model

In [ ]:
model = models.MLP(INPUT_SIZE, [128, 128, 128],
            dropout=0.05)
model.load_state_dict(torch.load(DD1S_CAIX_RANDOM_SPLIT_FP_FFNN_SEED_2_MODEL_PATH))
if DEVICE:
    model = model.to(DEVICE)

## Bit analysis

### Calculate bit weights

In [ ]:
set_bit_ids = [bit_id for bit_id in tqdm(range(2048)) if any(x[:,bit_id]==1)]
print(f'Number of bits set by at least one molecule in the data set: {len(set_bit_ids)}')

In [ ]:
basePreds_all = np.array(model.predict_on_x(x, device=DEVICE), dtype='float64')
if base_bit in set_bit_ids:
    set_bit_ids.remove(base_bit)
bit_to_weights = {bit_id: GetWeightsForTwoBits(base_bit, bit_id) for bit_id in tqdm(set_bit_ids)}
hkl.dump(bit_to_weights, 'twobits_bit_to_weights_DD1S_CAIX_FP-FFNN_random_seed_2.hkl', mode='w')

In [ ]:
# bit_to_weights_hkl = hkl.load('twobits_bit_to_weights_DD1S_CAIX_FP-FFNN_random_seed_2.hkl')
# bit_to_weights = {int(bit): weights for bit, weights in bit_to_weights_hkl.items()}

### Plot distribution of average bit weights

In [ ]:
avg_bit_weights = [bit_to_weights[b][1] for b in bit_to_weights.keys()]
print(f'Lowest average bit weight: {min(avg_bit_weights)}')
print(f'Highest average bit weight: {max(avg_bit_weights)}')

In [ ]:
# histogram of average bit weights (only including bits set by at least one molecule)
def make_hist_avg_bit_weights(zoomIn=False):
    fig = plt.figure(figsize=(3.5, 1.6), dpi=300)
    if zoomIn:
        bins = np.arange(0.078, 1.047, 0.01)
    else:
        bins = np.arange(0.078, 1.047, 0.02)
    _, bins, patches = plt.hist(
        np.clip(avg_bit_weights, 0.078, bins[-1]), 
        bins=bins,  
        density=False,
        zorder=2
    )
    fig.canvas.draw()
    ax = plt.gca()
    if zoomIn:
        ax.set_ylim([0, 8])
    ax.grid(zorder=1)
    ax.tick_params(labelsize=8)
    ax.set_xlabel('average weight', fontsize=8)
    ax.set_ylabel('number of bits', fontsize=8)
    plt.tight_layout()
    if zoomIn:
        plt.savefig(pathify(f'twobits_bit_weight_histogram_zoomed_in_DD1S_CAIX_FP-FFNN_random_seed_2.png'))
    else:
        plt.savefig(pathify(f'twobits_bit_weight_histogram_DD1S_CAIX_FP-FFNN_random_seed_2.png'))
    plt.show()

In [ ]:
make_hist_avg_bit_weights()

In [ ]:
make_hist_avg_bit_weights(zoomIn=True)

### Get bits of interest (based on average bit weight)

In [ ]:
bits_sorted_by_avg_weight = sorted(set_bit_ids, key = lambda b: bit_to_weights[b][1])
top_bits = bits_sorted_by_avg_weight[-5:]
top_bits.reverse()
bottom_bits = bits_sorted_by_avg_weight[:3]
bottom_bits.reverse()
print(f'Top bits: {top_bits}')
print(f'Bottom bits: {bottom_bits}')

In [ ]:
bits_of_interest = top_bits + bottom_bits
for b in bits_of_interest:
    print(f'Bit ID: {b}')
    print(f'Average weight: {bit_to_weights[b][1]}')
    mols_withbit = np.squeeze(np.where(x[:,b]==1))
    if mols_withbit.ndim > 0:
        print(f'Number of molecules with the bit: {len(mols_withbit)}')
    else:
        print('Number of molecules with the bit: 1')
    print()

In [ ]:
bit_to_cpd_row_indices = {
    1127: list([idx for idx in np.squeeze(np.where(x[:,1127]==1))]), 
    670: list([idx for idx in np.squeeze(np.where(x[:,670]==1))]), 
    1605: np.where(x[:,1605]==1), 
    1802: list([idx for idx in np.squeeze(np.where(x[:,1802]==1))]), 
    1262: list([idx for idx in np.squeeze(np.where(x[:,1262]==1))]), 
    1844: list([idx for idx in np.squeeze(np.where(x[:,1844]==1))]), 
    1523: list([idx for idx in np.squeeze(np.where(x[:,1523]==1))]), 
    1317: list([idx for idx in np.squeeze(np.where(x[:,1317]==1))]),
}
for item in bit_to_cpd_row_indices.items():
    print(f'Bit ID: {item[0]}')
    print(f'Number of molecules with the bit: {len(item[1])}')
    print()

### Plot distributions of molecule-level bit weights

In [ ]:
# histogram of molecule-level bit weights
def make_hist_mol_level_bit_weights(bit_id, x_lb, x_ub, stepsize_noZoom, stepsize_zoom, full=True, 
                                    zoomIn=False, zoomIn_y_ub=None, xticks=None, yticks=None):
    weights = bit_to_weights[bit_id][0]
    if full:
        fig = plt.figure(figsize=(3.5, 1.6), dpi=300)
    else:
        fig = plt.figure(figsize=(1.85, 1.6), dpi=300)
    if zoomIn:
        bins = np.arange(x_lb, x_ub, stepsize_zoom)
    else:
        bins = np.arange(x_lb, x_ub, stepsize_noZoom) 
    _, bins, patches = plt.hist(
        np.clip(weights, x_lb, bins[-1]), 
        bins=bins,  
        density=False,
        zorder=2
    )
    fig.canvas.draw()
    ax = plt.gca()
    if zoomIn:
        ax.set_ylim([0, zoomIn_y_ub])
    if xticks:
        plt.xticks(xticks)
    if yticks:
        plt.yticks(yticks)
    ax.grid(zorder=1)
    ax.tick_params(labelsize=8)
    ax.set_xlabel('weight', fontsize=8)
    ax.set_ylabel('number of molecules', fontsize=8)
    ax.set_title(f'bit {bit_id}', fontsize=8)
    plt.tight_layout()
    if zoomIn:
        plt.savefig(pathify(f'twobits_mol-level_bit_weight_histogram_bit_{bit_id}_zoomed_in_DD1S_CAIX_FP-FFNN_random_seed_2.png'))
    else:
        plt.savefig(pathify(f'twobits_mol-level_bit_weight_histogram_bit_{bit_id}_DD1S_CAIX_FP-FFNN_random_seed_2.png'))
    plt.show()

In [ ]:
bit = 1127
print(f'Bit {bit}')
print(f'Lowest molecule-level bit weight: {min(bit_to_weights[bit][0])}')
print(f'Highest molecule-level bit weight: {max(bit_to_weights[bit][0])}')

make_hist_mol_level_bit_weights(bit, 1.0296, 1.0572, 0.0005, None, full=True)

In [ ]:
bit = 670
print(f'Bit {bit}')
print(f'Lowest molecule-level bit weight: {min(bit_to_weights[bit][0])}')
print(f'Highest molecule-level bit weight: {max(bit_to_weights[bit][0])}')

make_hist_mol_level_bit_weights(bit, 1.00743, 1.0591, 0.0009, None, full=True, yticks=[0,1])

In [ ]:
bit = 1605
print(f'Bit {bit}')
print(f'Molecule-level bit weight: {bit_to_weights[bit][0]}')

make_hist_mol_level_bit_weights(bit, 1.0157, 1.0165, 0.000015, None, full=True, yticks=[0,1])

In [ ]:
bit = 1802
print(f'Bit {bit}')
print(f'Lowest molecule-level bit weight: {min(bit_to_weights[bit][0])}')
print(f'Highest molecule-level bit weight: {max(bit_to_weights[bit][0])}')

make_hist_mol_level_bit_weights(bit, 0.985, 1.0195, 0.00051, None, full=True, xticks=[0.985, 0.995, 1.005, 1.015])

In [ ]:
bit = 1262
print(f'Bit {bit}')
print(f'Lowest molecule-level bit weight: {min(bit_to_weights[bit][0])}')
print(f'Highest molecule-level bit weight: {max(bit_to_weights[bit][0])}')

make_hist_mol_level_bit_weights(bit, 0.958, 1.00630, 0.0009, None, full=True)

In [ ]:
bit = 1844
print(f'Bit {bit}')
print(f'Lowest molecule-level bit weight: {min(bit_to_weights[bit][0])}')
print(f'Highest molecule-level bit weight: {max(bit_to_weights[bit][0])}')

make_hist_mol_level_bit_weights(bit, 0.0844, 0.0948, 0.00018, None, full=True, yticks=[0,1])

In [ ]:
bit = 1523
print(f'Bit {bit}')
print(f'Lowest molecule-level bit weight: {min(bit_to_weights[bit][0])}')
print(f'Highest molecule-level bit weight: {max(bit_to_weights[bit][0])}')

make_hist_mol_level_bit_weights(bit, 0.0599, 0.103, 0.0008, None, full=True)

In [ ]:
bit = 1317
print(f'Bit {bit}')
print(f'Lowest molecule-level bit weight: {min(bit_to_weights[bit][0])}')
print(f'Highest molecule-level bit weight: {max(bit_to_weights[bit][0])}')

make_hist_mol_level_bit_weights(bit, 0.0699, 0.0870, 0.0003, None, full=True, yticks=[0,1], 
                                xticks=[0.0700, 0.0750, 0.0800, 0.0850])

### Get and visualize substructures

In [ ]:
bits_to_draw = []
submol_freq_distrib = {bit: {} for bit in bits_of_interest} # store frequency distribution of substructures that 
                                                            # set each bit
smarts_to_smis = {}
submol_to_cpd_indices = {} # mapping to indices in df_data_hasbit
submol_to_bit = {}
examples_all = {bit: {} for bit in bits_of_interest} # check if there's more than one distinct bit-setting 
                                                     # substructure in the same molecule

for bit in tqdm(bits_of_interest):
    df_data_hasbit = df_data.iloc[bit_to_cpd_row_indices[bit]]
    smis = df_data_hasbit['smiles']
    featurizer = featurizers.FingerprintFeaturizer()
    _, info_all = featurizer.prepare_x(df_data_hasbit, bitInfo=True)
    for i, smi in enumerate(smis):
        mol = Chem.MolFromSmiles(smi)
        atomSymbols = getMorganFingerprintAtomSymbols(mol)
        cpd_id = int(df_data[df_data['smiles']==smi]['cpd_id'].to_numpy()[0])
        if bit not in info_all[i]:
            continue
        fragment_logs = getFragmentForMolBit(smi, mol, i, atomSymbols, cpd_id, bit, info_all, submol_freq_distrib, 
                        smarts_to_smis, submol_to_cpd_indices, submol_to_bit, bits_to_draw)
        submol_freq_distrib = fragment_logs[0]
        smarts_to_smis = fragment_logs[1]
        submol_to_cpd_indices = fragment_logs[2]
        submol_to_bit = fragment_logs[3]
        bits_to_draw = fragment_logs[4]
        examples = fragment_logs[5]
        if examples:
            examples_all[bit][i] = examples

In [ ]:
hkl.dump(submol_freq_distrib, 'twobits_submol_freq_distrib_DD1S_CAIX_FP-FFNN_random_seed_2.hkl', mode='w')
hkl.dump(smarts_to_smis, 'twobits_smarts_to_smis_DD1S_CAIX_FP-FFNN_random_seed_2.hkl', mode='w')
hkl.dump(submol_to_cpd_indices, 'twobits_submol_to_cpd_indices_DD1S_CAIX_FP-FFNN_random_seed_2.hkl', mode='w')
hkl.dump(submol_to_bit, 'twobits_submol_to_bit_DD1S_CAIX_FP-FFNN_random_seed_2.hkl', mode='w')
hkl.dump(bits_to_draw, 'twobits_bits_to_draw_DD1S_CAIX_FP-FFNN_random_seed_2.hkl', mode='w')
hkl.dump(examples_all, 'twobits_examples_all_DD1S_CAIX_FP-FFNN_random_seed_2.hkl', mode='w')

In [ ]:
# submol_freq_distrib_hkl = hkl.load('twobits_submol_freq_distrib_DD1S_CAIX_FP-FFNN_random_seed_2.hkl')
# submol_freq_distrib = {int(bit): d for bit, d in submol_freq_distrib_hkl.items()}
# smarts_to_smis = hkl.load('twobits_smarts_to_smis_DD1S_CAIX_FP-FFNN_random_seed_2.hkl')
# submol_to_cpd_indices = hkl.load('twobits_submol_to_cpd_indices_DD1S_CAIX_FP-FFNN_random_seed_2.hkl')
# submol_to_bit = hkl.load('twobits_submol_to_bit_DD1S_CAIX_FP-FFNN_random_seed_2.hkl')
# bits_to_draw_hkl = hkl.load('twobits_bits_to_draw_DD1S_CAIX_FP-FFNN_random_seed_2.hkl')
# bits_to_draw = [(int(item[0]), Chem.MolFromSmiles(df_data_orig.iloc[int(item[0])-1]['smiles']), int(item[2]), 
#         {int(bit): tuple([(int(an[0]), int(an[1])) for an in ans]) for bit, ans in item[3].items()}) for item in bits_to_draw_hkl]
# examples_all_hkl = hkl.load('twobits_examples_all_DD1S_CAIX_FP-FFNN_random_seed_2.hkl')
# examples_all = {int(bit): {int(mol_idx): [int(ex_num) for ex_num in ex_nums] for mol_idx, ex_nums in exs.items()} for bit, exs in examples_all_hkl.items()}

In [ ]:
examples_all

In [ ]:
for item in smarts_to_smis.items():
    print(f'SMARTS: {item[0]}')
    print(f'SMILES: {np.squeeze(item[1])}')
    print()
print()
print(f'Number of distinct SMARTS: {len(set(smarts_to_smis.keys()))}')
print(f'Number of distinct SMILES: {len(set([j for i in smarts_to_smis.values() for j in i]))}')

In [ ]:
for bit in submol_freq_distrib:
    print(f'Bit ID: {bit}')
    for submol in submol_freq_distrib[bit]:
        print(f'Substructure (SMARTS): {submol}')
        print(f'Frequency: {submol_freq_distrib[bit][submol]}')
    print()

In [ ]:
_bits_to_draw = [bit[1:] for bit in bits_to_draw]
d = Draw.DrawMorganBits(_bits_to_draw, molsPerRow=4, aromaticColor=None, ringColor=None, 
                        legends=[f'cpd_id {bit[0]}, bit: {bit[2]}' for bit in bits_to_draw], subImgSize=(600, 600))
d.save(pathify(f'twobits_visualization_DD1S_CAIX_FP-FFNN_random_seed_2.png'))
d

## Substructure analysis

### Number substructures

In [ ]:
# numbering correponds to ordering of bars from left to right on plot of substructure weights (see below)
ctr = 1
submol_to_id = {}
for bit in bits_of_interest:
    submols = sorted(submol_freq_distrib[bit].keys(), key = lambda x: submol_freq_distrib[bit][x], reverse=True)
    for s in submols:
        submol_to_id[s] = ctr
        ctr += 1

In [ ]:
hkl.dump(submol_to_id, 'twobits_submol_to_id_DD1S_CAIX_FP-FFNN_random_seed_2.hkl', mode='w')

In [ ]:
# submol_to_id = hkl.load('twobits_submol_to_id_DD1S_CAIX_FP-FFNN_random_seed_2.hkl')

In [ ]:
for item in sorted(submol_to_id.items(), key=lambda x: x[1]):
    print(item[1])
    print(item[0])
    print(np.squeeze(smarts_to_smis[item[0]]))
    print()

### Calculate substructure weights

In [ ]:
substruct_to_weight = {}
for submol in submol_to_cpd_indices:
    bit = int(np.squeeze(submol_to_bit[submol]))
    if isinstance(bit_to_weights[bit][0], np.ndarray):
        substruct_to_weight[submol] = GetWeightsForSubstructure(bit_to_weights[bit][0], list(set(submol_to_cpd_indices[submol])))[1]
    else:
        substruct_to_weight[submol] = GetWeightsForSubstructure([bit_to_weights[bit][0]], 0)[1]

In [ ]:
for item in sorted(substruct_to_weight.items(), key=lambda x: submol_to_id[x[0]]):
    print(f'Substructure (SMARTS): {item[0]}')
    print()
    print(f'Weight: {item[1]}')
    print()
    print()

### Plot substructure weights

In [ ]:
substructs = sorted(substruct_to_weight.keys(), key=lambda s: submol_to_id[s])
weights = [substruct_to_weight[s] for s in substructs]
bars = []
for bit in bits_of_interest:
    _bars = []
    substruct_smarts_for_bit = submol_freq_distrib[bit].keys()
    for s in substructs:
        if s in substruct_smarts_for_bit:
            _bars.append(substruct_to_weight[s])
    bars.append(_bars)

barWidth = 2
offset = 2.2
space = 4

pos = len(bars[0])*barWidth / 2

In [ ]:
r1 = [pos]
pos += space + 1.5*barWidth + (offset-barWidth)
r2 = [pos]
pos += space + 1.5*barWidth + (offset-barWidth)
r3 = [pos]
pos += space + 1.5*barWidth + (offset-barWidth)
r4 = [pos]
pos += space + 1.5*barWidth + (offset-barWidth)
r5 = [pos - 0.5*barWidth - 0.5*(offset-barWidth), pos + 0.5*barWidth + 0.5*(offset-barWidth)]
pos += space + 1.5*barWidth + (offset-barWidth)
r6 = [pos]
pos += space + 1.5*barWidth + (offset-barWidth)
r7 = [pos]
pos += space + 1.5*barWidth + (offset-barWidth)
r8 = [pos]
rs = [r1, r2, r3, r4, r5, r6, r7, r8]

In [ ]:
fig = plt.figure(figsize=(7, 4), dpi=300)

rects = []
for bit in range(len(bars)):
    for i in range(len(rs[bit])):
        rects.append(plt.bar(rs[bit][i], bars[bit][i], color='#1f77b4', width=barWidth, zorder=2)[0])
        
for rect in rects:
    height = rect.get_height()
    plt.annotate('{:.3f}'.format(height),
                xy=(rect.get_x() + rect.get_width() / 2, height),
                xytext=(0, 3),  
                textcoords="offset points",
                ha='center', va='bottom',
                fontsize=4.5)
        
fig.canvas.draw()
ax = plt.gca()
ax.grid(zorder=1)
ax.set_xlabel('bit ID', fontsize=9)
ax.set_ylabel('substructure weight', fontsize=9)
ax.tick_params(labelsize=9)
ax.set_xticks([((len(bars[0])*barWidth / 2) + i*(space + 1.5*barWidth + (offset-barWidth))) for i in range(len(bars))])
ax.set_xticklabels([str(bit_id) for bit_id in bits_of_interest], ha='center')
ax.tick_params(axis='x', length=0)
plt.tight_layout()
plt.savefig(pathify(f'twobits_substructure_weights_DD1S_CAIX_FP-FFNN_random_seed_2.png'))
plt.show()

# Triazine sEH (random split, seed 0)

## Load data

In [ ]:
os.environ["HDF5_USE_FILE_LOCKING"] = 'FALSE'
df_data_orig = pd.read_csv(os.path.join(DELQSAR_ROOT, 'experiments', 'datasets', 'triazine_lib_sEH_SIRT2_QSAR.csv'))
hf = h5py.File(os.path.join(DELQSAR_ROOT, 'experiments', triazine_FINGERPRINTS_FILENAME))
x = np.array(hf['all_fps'])
INPUT_SIZE = x.shape[1] 
hf.close()

In [ ]:
# reduce to molecules with top substructure
submol = triazine_sEH_SEED_0_TOP_SUBSTRUCTURE_SMARTS
base_bit = triazine_sEH_SEED_0_BASE_BIT
cpd_row_indices_for_bit = [idx for idx in np.squeeze(np.where(x[:,base_bit]==1))]
submol_to_cpd_indices = hkl.load('submol_to_cpd_indices_triazine_sEH_FP-FFNN_random_seed_0.hkl') # getting substructure SMARTS and cpd indices from single bit substructure analysis
x_withsubmol_rows = [cpd_row_indices_for_bit[i] for i in list(set(submol_to_cpd_indices[submol]))]

df_data = df_data_orig.iloc[x_withsubmol_rows]
x = x[x_withsubmol_rows, :]

## Load model

In [ ]:
model = models.MLP(INPUT_SIZE, [256, 128, 64],
            dropout=0.4)
model.load_state_dict(torch.load(triazine_sEH_RANDOM_SPLIT_FP_FFNN_SEED_0_MODEL_PATH))
if DEVICE:
    model = model.to(DEVICE)

## Bit analysis

### Calculate bit weights

In [ ]:
set_bit_ids = [bit_id for bit_id in tqdm(range(2048)) if any(x[:,bit_id]==1)]
print(f'Number of bits set by at least one molecule in the data set: {len(set_bit_ids)}')

In [ ]:
basePreds_all = np.array(model.predict_on_x(x, device=DEVICE), dtype='float64')
if base_bit in set_bit_ids:
    set_bit_ids.remove(base_bit)
bit_to_weights = {bit_id: GetWeightsForTwoBits(base_bit, bit_id) for bit_id in tqdm(set_bit_ids)}
hkl.dump(bit_to_weights, 'twobits_bit_to_weights_triazine_sEH_FP-FFNN_random_seed_0.hkl', mode='w')

In [ ]:
# bit_to_weights_hkl = hkl.load('twobits_bit_to_weights_triazine_sEH_FP-FFNN_random_seed_0.hkl')
# bit_to_weights = {int(bit): weights for bit, weights in bit_to_weights_hkl.items()}

###  Plot distribution of average bit weights

In [ ]:
avg_bit_weights = [bit_to_weights[b][1] for b in bit_to_weights.keys()]
print(f'Lowest average bit weight: {min(avg_bit_weights)}')
print(f'Highest average bit weight: {max(avg_bit_weights)}')

In [ ]:
# histogram of average bit weights (only including bits set by at least one molecule)
def make_hist_avg_bit_weights(zoomIn=False):
    fig = plt.figure(figsize=(3.5, 1.6), dpi=300)
    if zoomIn:
        bins = np.arange(-16.77, 69.26, 0.75)
    else:
        bins = np.arange(-16.77, 69.26, 0.8)
    _, bins, patches = plt.hist(
        np.clip(avg_bit_weights, -16.77, bins[-1]), 
        bins=bins,  
        density=False,
        zorder=2
    )
    fig.canvas.draw()
    ax = plt.gca()
    if zoomIn:
        ax.set_ylim([0, 8])
    ax.grid(zorder=1)
    ax.tick_params(labelsize=8)
    ax.set_xlabel('average weight', fontsize=8)
    ax.set_ylabel('number of bits', fontsize=8)
    plt.tight_layout()
    if zoomIn:
        plt.savefig(pathify(f'twobits_bit_weight_histogram_zoomed_in_triazine_sEH_FP-FFNN_random_seed_0.png'))
    else:
        plt.savefig(pathify(f'twobits_bit_weight_histogram_triazine_sEH_FP-FFNN_random_seed_0.png'))
    plt.show()

In [ ]:
make_hist_avg_bit_weights()

In [ ]:
make_hist_avg_bit_weights(zoomIn=True)

### Get bits of interest (based on average bit weight)

In [ ]:
bits_sorted_by_avg_weight = sorted(set_bit_ids, key = lambda b: bit_to_weights[b][1])
top_bits = bits_sorted_by_avg_weight[-5:]
top_bits.reverse()
bottom_bits = bits_sorted_by_avg_weight[:3]
bottom_bits.reverse()
print(f'Top bits: {top_bits}')
print(f'Bottom bits: {bottom_bits}')

In [ ]:
bits_of_interest = top_bits + bottom_bits
for b in bits_of_interest:
    print(f'Bit ID: {b}')
    print(f'Average weight: {bit_to_weights[b][1]}')
    print(f'Number of molecules with the bit: {len(np.squeeze(np.where(x[:,b]==1)))}')
    print()

In [ ]:
bit_to_cpd_row_indices = {bit: list([idx for idx in np.squeeze(np.where(x[:,bit]==1))]) for bit in bits_of_interest}
for item in bit_to_cpd_row_indices.items():
    print(f'Bit ID: {item[0]}')
    print(f'Number of molecules with the bit: {len(item[1])}')
    print()

### Plot distributions of molecule-level bit weights

In [ ]:
# histogram of molecule-level bit weights
def make_hist_mol_level_bit_weights(bit_id, x_lb, x_ub, stepsize_noZoom, stepsize_zoom, zoomIn=False,
                                    zoomIn_y_ub=None, xticks=None, yticks=None):
    weights = bit_to_weights[bit_id][0]
    fig = plt.figure(figsize=(3.5, 1.6), dpi=300)
    if zoomIn:
        bins = np.arange(x_lb, x_ub, stepsize_zoom)
    else:
        bins = np.arange(x_lb, x_ub, stepsize_noZoom) 
    _, bins, patches = plt.hist(
        np.clip(weights, x_lb, bins[-1]), 
        bins=bins,  
        density=False,
        zorder=2
    )
    fig.canvas.draw()
    ax = plt.gca()
    if zoomIn:
        ax.set_ylim([0, zoomIn_y_ub])
    ax.grid(zorder=1)
    ax.tick_params(labelsize=8)
    if xticks:
        plt.xticks(xticks)
    if yticks:
        plt.yticks(yticks)
    ax.set_xlabel('weight', fontsize=8)
    ax.set_ylabel('number of molecules', fontsize=8)
    ax.set_title(f'bit {bit_id}', fontsize=8)
    plt.tight_layout()
    if zoomIn:
        plt.savefig(pathify(f'twobits_mol-level_bit_weight_histogram_bit_{bit_id}_zoomed_in_triazine_sEH_FP-FFNN_random_seed_0.png'))
    else:
        plt.savefig(pathify(f'twobits_mol-level_bit_weight_histogram_bit_{bit_id}_triazine_sEH_FP-FFNN_random_seed_0.png'))
    plt.show()

In [ ]:
bit = 1402
print(f'Bit {bit}')
print(f'Lowest molecule-level bit weight: {min(bit_to_weights[bit][0])}')
print(f'Highest molecule-level bit weight: {max(bit_to_weights[bit][0])}')

make_hist_mol_level_bit_weights(bit, 12.680, 85.409, 1.5, None)
                                

In [ ]:
bit = 1554
print(f'Bit {bit}')
print(f'Lowest molecule-level bit weight: {min(bit_to_weights[bit][0])}')
print(f'Highest molecule-level bit weight: {max(bit_to_weights[bit][0])}')

make_hist_mol_level_bit_weights(bit, 12.541, 85.383, 1.6, None)

In [ ]:
bit = 1198
print(f'Bit {bit}')
print(f'Lowest molecule-level bit weight: {min(bit_to_weights[bit][0])}')
print(f'Highest molecule-level bit weight: {max(bit_to_weights[bit][0])}')

make_hist_mol_level_bit_weights(bit, 11.636, 84.733, 1.6, None)

In [ ]:
bit = 1245
print(f'Bit {bit}')
print(f'Lowest molecule-level bit weight: {min(bit_to_weights[bit][0])}')
print(f'Highest molecule-level bit weight: {max(bit_to_weights[bit][0])}')

make_hist_mol_level_bit_weights(bit, 0, 82.660, 1.7, None)

In [ ]:
bit = 1368
print(f'Bit {bit}')
print(f'Lowest molecule-level bit weight: {min(bit_to_weights[bit][0])}')
print(f'Highest molecule-level bit weight: {max(bit_to_weights[bit][0])}')

make_hist_mol_level_bit_weights(bit, 0, 85.920, 1.68, None)

In [ ]:
bit = 1923
print(f'Bit {bit}')
print(f'Lowest molecule-level bit weight: {min(bit_to_weights[bit][0])}')
print(f'Highest molecule-level bit weight: {max(bit_to_weights[bit][0])}')

make_hist_mol_level_bit_weights(bit, -27.184, 1.398, 0.4, None)

In [ ]:
bit = 853
print(f'Bit {bit}')
print(f'Lowest molecule-level bit weight: {min(bit_to_weights[bit][0])}')
print(f'Highest molecule-level bit weight: {max(bit_to_weights[bit][0])}')

make_hist_mol_level_bit_weights(bit, -49.422, 0, 0.8, None)

In [ ]:
bit = 864
print(f'Bit {bit}')
print(f'Lowest molecule-level bit weight: {min(bit_to_weights[bit][0])}')
print(f'Highest molecule-level bit weight: {max(bit_to_weights[bit][0])}')

make_hist_mol_level_bit_weights(bit, -53.0481, 1.906, 1.2, None)

### Get and visualize substructures

In [ ]:
bits_to_draw = []
submol_freq_distrib = {bit: {} for bit in bits_of_interest} # store frequency distribution of substructures that 
                                                            # set each bit
smarts_to_smis = {}
submol_to_cpd_indices = {} # mapping to indices in df_data_hasbit
submol_to_bit = {}
examples_all = {bit: {} for bit in bits_of_interest} # check if there's more than one distinct bit-setting 
                                                     # substructure in the same molecule

for bit in tqdm(bits_of_interest):
    df_data_hasbit = df_data.iloc[bit_to_cpd_row_indices[bit]]
    smis = df_data_hasbit['smiles']
    featurizer = featurizers.FingerprintFeaturizer()
    _, info_all = featurizer.prepare_x(df_data_hasbit, bitInfo=True)
    for i, smi in enumerate(smis):
        mol = Chem.MolFromSmiles(smi)
        atomSymbols = getMorganFingerprintAtomSymbols(mol)
        cpd_id = int(df_data[df_data['smiles']==smi]['cpd_id'].to_numpy()[0])
        if bit not in info_all[i]:
            continue
        fragment_logs = getFragmentForMolBit(smi, mol, i, atomSymbols, cpd_id, bit, info_all, submol_freq_distrib, 
                        smarts_to_smis, submol_to_cpd_indices, submol_to_bit, bits_to_draw)
        submol_freq_distrib = fragment_logs[0]
        smarts_to_smis = fragment_logs[1]
        submol_to_cpd_indices = fragment_logs[2]
        submol_to_bit = fragment_logs[3]
        bits_to_draw = fragment_logs[4]
        examples = fragment_logs[5]
        if examples:
            examples_all[bit][i] = examples

In [ ]:
hkl.dump(submol_freq_distrib, 'twobits_submol_freq_distrib_triazine_sEH_FP-FFNN_random_seed_0.hkl', mode='w')
hkl.dump(smarts_to_smis, 'twobits_smarts_to_smis_triazine_sEH_FP-FFNN_random_seed_0.hkl', mode='w')
hkl.dump(submol_to_cpd_indices, 'twobits_submol_to_cpd_indices_triazine_sEH_FP-FFNN_random_seed_0.hkl', mode='w')
hkl.dump(submol_to_bit, 'twobits_submol_to_bit_triazine_sEH_FP-FFNN_random_seed_0.hkl', mode='w')
hkl.dump(bits_to_draw, 'twobits_bits_to_draw_triazine_sEH_FP-FFNN_random_seed_0.hkl', mode='w')
hkl.dump(examples_all, 'twobits_examples_all_triazine_sEH_FP-FFNN_random_seed_0.hkl', mode='w')

In [ ]:
# submol_freq_distrib_hkl = hkl.load('twobits_submol_freq_distrib_triazine_sEH_FP-FFNN_random_seed_0.hkl')
# submol_freq_distrib = {int(bit): d for bit, d in submol_freq_distrib_hkl.items()}
# smarts_to_smis = hkl.load('twobits_smarts_to_smis_triazine_sEH_FP-FFNN_random_seed_0.hkl')
# submol_to_cpd_indices = hkl.load('twobits_submol_to_cpd_indices_triazine_sEH_FP-FFNN_random_seed_0.hkl')
# submol_to_bit = hkl.load('twobits_submol_to_bit_triazine_sEH_FP-FFNN_random_seed_0.hkl')
# bits_to_draw_hkl = hkl.load('twobits_bits_to_draw_triazine_sEH_FP-FFNN_random_seed_0.hkl')
# bits_to_draw = [(int(item[0]), Chem.MolFromSmiles(df_data_orig.iloc[int(item[0])-1]['smiles']), int(item[2]), 
#         {int(bit): tuple([(int(an[0]), int(an[1])) for an in ans]) for bit, ans in item[3].items()}) for item in bits_to_draw_hkl]
# examples_all_hkl = hkl.load('twobits_examples_all_triazine_sEH_FP-FFNN_random_seed_0.hkl')
# examples_all = {int(bit): {int(mol_idx): [int(ex_num) for ex_num in ex_nums] for mol_idx, ex_nums in exs.items()} for bit, exs in examples_all_hkl.items()}

In [ ]:
examples_all

In [ ]:
for item in smarts_to_smis.items():
    print(f'SMARTS: {item[0]}')
    print(f'SMILES: {np.squeeze(item[1])}')
    print()
print()
print(f'Number of distinct SMARTS: {len(set(smarts_to_smis.keys()))}')
print(f'Number of distinct SMILES: {len(set([j for i in smarts_to_smis.values() for j in i]))}')

In [ ]:
for bit in submol_freq_distrib:
    print(f'Bit ID: {bit}')
    for submol in submol_freq_distrib[bit]:
        print(f'Substructure (SMARTS): {submol}')
        print(f'Frequency: {submol_freq_distrib[bit][submol]}')
    print()

In [ ]:
_bits_to_draw = [bit[1:] for bit in bits_to_draw]

In [ ]:
d = Draw.DrawMorganBits(_bits_to_draw, molsPerRow=4, aromaticColor=None, ringColor=None, 
                        legends=[f'cpd_id {bit[0]}, bit: {bit[2]}' for bit in bits_to_draw], subImgSize=(600, 600))
d.save(pathify(f'twobits_visualization_triazine_sEH_FP-FFNN_random_seed_0.png'))
d

## Substructure analysis

### Number substructures

In [ ]:
# numbering correponds to ordering of bars from left to right on plot of substructure weights (see below)
ctr = 1
submol_to_id = {}
for bit in bits_of_interest:
    submols = sorted(submol_freq_distrib[bit].keys(), key = lambda x: submol_freq_distrib[bit][x], reverse=True)
    for s in submols:
        submol_to_id[s] = ctr
        ctr += 1

In [ ]:
hkl.dump(submol_to_id, 'twobits_submol_to_id_triazine_sEH_FP-FFNN_random_seed_0.hkl', mode='w')

In [ ]:
# submol_to_id = hkl.load('twobits_submol_to_id_triazine_sEH_FP-FFNN_random_seed_0.hkl')

In [ ]:
for item in sorted(submol_to_id.items(), key=lambda x: x[1]):
    print(item[1])
    print(item[0])
    print(np.squeeze(smarts_to_smis[item[0]]))
    print()

### Calculate substructure weights

In [ ]:
substruct_to_weight = {}
for submol in submol_to_cpd_indices:
    bit = int(np.squeeze(submol_to_bit[submol]))
    substruct_to_weight[submol] = GetWeightsForSubstructure(bit_to_weights[bit][0], list(set(submol_to_cpd_indices[submol])))[1]

In [ ]:
for item in sorted(substruct_to_weight.items(), key=lambda x: submol_to_id[x[0]]):
    print(f'Substructure (SMARTS): {item[0]}')
    print()
    print(f'Weight: {item[1]}')
    print()
    print()

### Plot substructure weights

In [ ]:
substructs = sorted(substruct_to_weight.keys(), key=lambda s: submol_to_id[s])
weights = [substruct_to_weight[s] for s in substructs]
bars = []
for bit in bits_of_interest:
    _bars = []
    substruct_smarts_for_bit = submol_freq_distrib[bit].keys()
    for s in substructs:
        if s in substruct_smarts_for_bit:
            _bars.append(substruct_to_weight[s])
    bars.append(_bars)

barWidth = 2
offset = 2.2
space = 4

pos = len(bars[0])*barWidth / 2

In [ ]:
r1 = [pos]
pos += space + 1.5*barWidth + (offset-barWidth)
r2 = [pos]
pos += space + 1.5*barWidth + (offset-barWidth)
r3 = [pos]
pos += space + 1.5*barWidth + (offset-barWidth)
r4 = [pos - 0.5*barWidth - 0.5*(offset-barWidth), pos + 0.5*barWidth + 0.5*(offset-barWidth)]
pos += space + 1.5*barWidth + (offset-barWidth)
r5 = [pos - 0.5*barWidth - 0.5*(offset-barWidth), pos + 0.5*barWidth + 0.5*(offset-barWidth)]
pos += space + 1.5*barWidth + (offset-barWidth)
r6 = [pos]
pos += space + 1.5*barWidth + (offset-barWidth)
r7 = [pos]
pos += space + 1.5*barWidth + (offset-barWidth)
r8 = [pos]
rs = [r1, r2, r3, r4, r5, r6, r7, r8]

In [ ]:
fig = plt.figure(figsize=(7, 4), dpi=300)

rects = []
for bit in range(len(bars)):
    for i in range(len(rs[bit])):
        rects.append(plt.bar(rs[bit][i], bars[bit][i], color='#1f77b4', width=barWidth, zorder=2)[0])

for rect in rects[:-3]:
    height = rect.get_height()
    plt.annotate('{:.3f}'.format(height),
                xy=(rect.get_x() + rect.get_width() / 2, height),
                xytext=(0, 3),  
                textcoords="offset points",
                ha='center', va='bottom',
                fontsize=4.5)
for rect in rects[-3:]:
    height = rect.get_height()
    plt.annotate('{:.3f}'.format(height),
                xy=(rect.get_x() + rect.get_width() / 2, height),
                xytext=(0, -8),  
                textcoords="offset points",
                ha='center', va='bottom',
                fontsize=4.5)

fig.canvas.draw()
ax = plt.gca()
ax.grid(zorder=1)
ax.set_xlabel('bit ID', fontsize=9)
ax.set_ylabel('substructure weight', fontsize=9)
ax.tick_params(labelsize=9)
ax.set_xticks([((len(bars[0])*barWidth / 2) + i*(space + 1.5*barWidth + (offset-barWidth))) for i in range(len(bars))])
ax.set_xticklabels([str(bit_id) for bit_id in bits_of_interest], ha='center')
ax.tick_params(axis='x', length=0)
plt.tight_layout()
plt.savefig(pathify(f'twobits_substructure_weights_triazine_sEH_FP-FFNN_random_seed_0.png'))
plt.show()

# Triazine SIRT2 (random split, seed 0)

## Load data

In [ ]:
os.environ["HDF5_USE_FILE_LOCKING"] = 'FALSE'
df_data_orig = pd.read_csv(os.path.join(DELQSAR_ROOT, 'experiments', 'datasets', 'triazine_lib_sEH_SIRT2_QSAR.csv'))
hf = h5py.File(os.path.join(DELQSAR_ROOT, 'experiments', triazine_FINGERPRINTS_FILENAME))
x = np.array(hf['all_fps'])
INPUT_SIZE = x.shape[1] 
hf.close()

In [ ]:
# reduce to molecules with top substructure
submol = triazine_SIRT2_SEED_0_TOP_SUBSTRUCTURE_SMARTS
base_bit = triazine_SIRT2_SEED_0_BASE_BIT
cpd_row_indices_for_bit = [idx for idx in np.squeeze(np.where(x[:,base_bit]==1))]
submol_to_cpd_indices = hkl.load('submol_to_cpd_indices_triazine_SIRT2_FP-FFNN_random_seed_0.hkl') # getting substructure SMARTS and cpd indices from single bit substructure analysis
x_withsubmol_rows = [cpd_row_indices_for_bit[i] for i in list(set(submol_to_cpd_indices[submol]))]

df_data = df_data_orig.iloc[x_withsubmol_rows]
x = x[x_withsubmol_rows, :]

## Load model

In [ ]:
model = models.MLP(INPUT_SIZE, [256, 128, 64],
            dropout=0.1)
model.load_state_dict(torch.load(triazine_SIRT2_RANDOM_SPLIT_FP_FFNN_SEED_0_MODEL_PATH))
if DEVICE:
    model = model.to(DEVICE)

## Bit analysis

### Calculate bit weights

In [ ]:
set_bit_ids = [bit_id for bit_id in tqdm(range(2048)) if any(x[:,bit_id]==1)]
print(f'Number of bits set by at least one molecule in the data set: {len(set_bit_ids)}')

In [ ]:
basePreds_all = np.array(model.predict_on_x(x, device=DEVICE), dtype='float64')
if base_bit in set_bit_ids:
    set_bit_ids.remove(base_bit)
bit_to_weights = {bit_id: GetWeightsForTwoBits(base_bit, bit_id) for bit_id in tqdm(set_bit_ids)}
hkl.dump(bit_to_weights, 'twobits_bit_to_weights_triazine_SIRT2_FP-FFNN_random_seed_0.hkl', mode='w')

In [ ]:
# bit_to_weights_hkl = hkl.load('twobits_bit_to_weights_triazine_SIRT2_FP-FFNN_random_seed_0.hkl')
# bit_to_weights = {int(bit): weights for bit, weights in bit_to_weights_hkl.items()}

### Plot distribution of average bit weights

In [ ]:
avg_bit_weights = [bit_to_weights[b][1] for b in bit_to_weights.keys()]
print(f'Lowest average bit weight: {min(avg_bit_weights)}')
print(f'Highest average bit weight: {max(avg_bit_weights)}')

In [ ]:
# histogram of average bit weights (only including bits set by at least one molecule)
def make_hist_avg_bit_weights(zoomIn=False):
    fig = plt.figure(figsize=(3.5, 1.6), dpi=300)
    if zoomIn:
        bins = np.arange(-1.02, 3.18, 0.05)
    else:
        bins = np.arange(-1.02, 3.18, 0.07)
    _, bins, patches = plt.hist(
        np.clip(avg_bit_weights, -1.02, bins[-1]), 
        bins=bins,  
        density=False,
        zorder=2
    )
    fig.canvas.draw()
    ax = plt.gca()
    if zoomIn:
        ax.set_ylim([0, 8])
    ax.grid(zorder=1)
    ax.tick_params(labelsize=8)
    ax.set_xlabel('average weight', fontsize=8)
    ax.set_ylabel('number of bits', fontsize=8)
    plt.tight_layout()
    if zoomIn:
        plt.savefig(pathify(f'twobits_bit_weight_histogram_zoomed_in_triazine_SIRT2_FP-FFNN_random_seed_0.png'))
    else:
        plt.savefig(pathify(f'twobits_bit_weight_histogram_triazine_SIRT2_FP-FFNN_random_seed_0.png'))
    plt.show()

In [ ]:
make_hist_avg_bit_weights()

In [ ]:
make_hist_avg_bit_weights(zoomIn=True)

### Get bits of interest (based on average bit weight)¶

In [ ]:
bits_sorted_by_avg_weight = sorted(set_bit_ids, key = lambda b: bit_to_weights[b][1])
top_bits = bits_sorted_by_avg_weight[-5:]
top_bits.reverse()
bottom_bits = bits_sorted_by_avg_weight[:3]
bottom_bits.reverse()
print(f'Top bits: {top_bits}')
print(f'Bottom bits: {bottom_bits}')

In [ ]:
bits_of_interest = top_bits + bottom_bits
for b in bits_of_interest:
    print(f'Bit ID: {b}')
    print(f'Average weight: {bit_to_weights[b][1]}')
    print(f'Number of molecules with the bit: {len(np.squeeze(np.where(x[:,b]==1)))}')
    print()

In [ ]:
bit_to_cpd_row_indices = {bit: list([idx for idx in np.squeeze(np.where(x[:,bit]==1))]) for bit in bits_of_interest}
for item in bit_to_cpd_row_indices.items():
    print(f'Bit ID: {item[0]}')
    print(f'Number of molecules with the bit: {len(item[1])}')
    print()

### Plot distributions of molecule-level bit weights

In [ ]:
# histogram of molecule-level bit weights
def make_hist_mol_level_bit_weights(bit_id, x_lb, x_ub, stepsize_noZoom, stepsize_zoom, zoomIn=False,
                                    zoomIn_y_ub=None, xticks=None, yticks=None):
    weights = bit_to_weights[bit_id][0]
    fig = plt.figure(figsize=(3.5, 1.6), dpi=300)
    if zoomIn:
        bins = np.arange(x_lb, x_ub, stepsize_zoom)
    else:
        bins = np.arange(x_lb, x_ub, stepsize_noZoom) 
    _, bins, patches = plt.hist(
        np.clip(weights, x_lb, bins[-1]), 
        bins=bins,  
        density=False,
        zorder=2
    )
    fig.canvas.draw()
    ax = plt.gca()
    if zoomIn:
        ax.set_ylim([0, zoomIn_y_ub])
    ax.grid(zorder=1)
    ax.tick_params(labelsize=8)
    if xticks:
        plt.xticks(xticks)
    if yticks:
        plt.yticks(yticks)
    ax.set_xlabel('weight', fontsize=8)
    ax.set_ylabel('number of molecules', fontsize=8)
    ax.set_title(f'bit {bit_id}', fontsize=8)
    plt.tight_layout()
    if zoomIn:
        plt.savefig(pathify(f'twobits_mol-level_bit_weight_histogram_bit_{bit_id}_zoomed_in_triazine_SIRT2_FP-FFNN_random_seed_0.png'))
    else:
        plt.savefig(pathify(f'twobits_mol-level_bit_weight_histogram_bit_{bit_id}_triazine_SIRT2_FP-FFNN_random_seed_0.png'))
    plt.show()

In [ ]:
bit = 1024
print(f'Bit {bit}')
print(f'Lowest molecule-level bit weight: {min(bit_to_weights[bit][0])}')
print(f'Highest molecule-level bit weight: {max(bit_to_weights[bit][0])}')

make_hist_mol_level_bit_weights(bit, 1.296, 3.716, 0.05, None)

In [ ]:
bit = 330
print(f'Bit {bit}')
print(f'Lowest molecule-level bit weight: {min(bit_to_weights[bit][0])}')
print(f'Highest molecule-level bit weight: {max(bit_to_weights[bit][0])}')

make_hist_mol_level_bit_weights(bit, -0.248, 4.602, 0.1, None)

In [ ]:
bit = 1643
print(f'Bit {bit}')
print(f'Lowest molecule-level bit weight: {min(bit_to_weights[bit][0])}')
print(f'Highest molecule-level bit weight: {max(bit_to_weights[bit][0])}')

make_hist_mol_level_bit_weights(bit, -0.206, 4.315, 0.095, None)

In [ ]:
bit = 1272
print(f'Bit {bit}')
print(f'Lowest molecule-level bit weight: {min(bit_to_weights[bit][0])}')
print(f'Highest molecule-level bit weight: {max(bit_to_weights[bit][0])}')

make_hist_mol_level_bit_weights(bit, -0.199, 4.397, 0.097, None)

In [ ]:
bit = 1404
print(f'Bit {bit}')
print(f'Lowest molecule-level bit weight: {min(bit_to_weights[bit][0])}')
print(f'Highest molecule-level bit weight: {max(bit_to_weights[bit][0])}')

make_hist_mol_level_bit_weights(bit, -0.00462, 3.000694, 0.063, None)

In [ ]:
bit = 121
print(f'Bit {bit}')
print(f'Lowest molecule-level bit weight: {min(bit_to_weights[bit][0])}')
print(f'Highest molecule-level bit weight: {max(bit_to_weights[bit][0])}')

make_hist_mol_level_bit_weights(bit, -0.797, 0, 0.017, None)

In [ ]:
bit = 1187
print(f'Bit {bit}')
print(f'Lowest molecule-level bit weight: {min(bit_to_weights[bit][0])}')
print(f'Highest molecule-level bit weight: {max(bit_to_weights[bit][0])}')

make_hist_mol_level_bit_weights(bit, -0.795, 0, 0.021, None)

In [ ]:
bit = 234
print(f'Bit {bit}')
print(f'Lowest molecule-level bit weight: {min(bit_to_weights[bit][0])}')
print(f'Highest molecule-level bit weight: {max(bit_to_weights[bit][0])}')

make_hist_mol_level_bit_weights(bit, -2.397, 0.0492, 0.05, None)

### Get and visualize substructures

In [ ]:
bits_to_draw = []
submol_freq_distrib = {bit: {} for bit in bits_of_interest} # store frequency distribution of substructures that 
                                                            # set each bit
smarts_to_smis = {}
submol_to_cpd_indices = {} # mapping to indices in df_data_hasbit
submol_to_bit = {}
examples_all = {bit: {} for bit in bits_of_interest} # check if there's more than one distinct bit-setting 
                                                     # substructure in the same molecule

for bit in tqdm(bits_of_interest):
    df_data_hasbit = df_data.iloc[bit_to_cpd_row_indices[bit]]
    smis = df_data_hasbit['smiles']
    featurizer = featurizers.FingerprintFeaturizer()
    _, info_all = featurizer.prepare_x(df_data_hasbit, bitInfo=True)
    for i, smi in enumerate(smis):
        mol = Chem.MolFromSmiles(smi)
        atomSymbols = getMorganFingerprintAtomSymbols(mol)
        cpd_id = int(df_data[df_data['smiles']==smi]['cpd_id'].to_numpy()[0])
        if bit not in info_all[i]:
            continue
        fragment_logs = getFragmentForMolBit(smi, mol, i, atomSymbols, cpd_id, bit, info_all, submol_freq_distrib, 
                        smarts_to_smis, submol_to_cpd_indices, submol_to_bit, bits_to_draw)
        submol_freq_distrib = fragment_logs[0]
        smarts_to_smis = fragment_logs[1]
        submol_to_cpd_indices = fragment_logs[2]
        submol_to_bit = fragment_logs[3]
        bits_to_draw = fragment_logs[4]
        examples = fragment_logs[5]
        if examples:
            examples_all[bit][i] = examples

In [ ]:
hkl.dump(submol_freq_distrib, 'twobits_submol_freq_distrib_triazine_SIRT2_FP-FFNN_random_seed_0.hkl', mode='w')
hkl.dump(smarts_to_smis, 'twobits_smarts_to_smis_triazine_SIRT2_FP-FFNN_random_seed_0.hkl', mode='w')
hkl.dump(submol_to_cpd_indices, 'twobits_submol_to_cpd_indices_triazine_SIRT2_FP-FFNN_random_seed_0.hkl', mode='w')
hkl.dump(submol_to_bit, 'twobits_submol_to_bit_triazine_SIRT2_FP-FFNN_random_seed_0.hkl', mode='w')
hkl.dump(bits_to_draw, 'twobits_bits_to_draw_triazine_SIRT2_FP-FFNN_random_seed_0.hkl', mode='w')
hkl.dump(examples_all, 'twobits_examples_all_triazine_SIRT2_FP-FFNN_random_seed_0.hkl', mode='w')

In [ ]:
# submol_freq_distrib_hkl = hkl.load('twobits_submol_freq_distrib_triazine_SIRT2_FP-FFNN_random_seed_0.hkl')
# submol_freq_distrib = {int(bit): d for bit, d in submol_freq_distrib_hkl.items()}
# smarts_to_smis = hkl.load('twobits_smarts_to_smis_triazine_SIRT2_FP-FFNN_random_seed_0.hkl')
# submol_to_cpd_indices = hkl.load('twobits_submol_to_cpd_indices_triazine_SIRT2_FP-FFNN_random_seed_0.hkl')
# submol_to_bit = hkl.load('twobits_submol_to_bit_triazine_SIRT2_FP-FFNN_random_seed_0.hkl')
# bits_to_draw_hkl = hkl.load('twobits_bits_to_draw_triazine_SIRT2_FP-FFNN_random_seed_0.hkl')
# bits_to_draw = [(int(item[0]), Chem.MolFromSmiles(df_data_orig.iloc[int(item[0])-1]['smiles']), int(item[2]), 
#         {int(bit): tuple([(int(an[0]), int(an[1])) for an in ans]) for bit, ans in item[3].items()}) for item in bits_to_draw_hkl]
# examples_all_hkl = hkl.load('twobits_examples_all_triazine_SIRT2_FP-FFNN_random_seed_0.hkl')
# examples_all = {int(bit): {int(mol_idx): [int(ex_num) for ex_num in ex_nums] for mol_idx, ex_nums in exs.items()} for bit, exs in examples_all_hkl.items()}

In [ ]:
examples_all

In [ ]:
for item in smarts_to_smis.items():
    print(f'SMARTS: {item[0]}')
    print(f'SMILES: {np.squeeze(item[1])}')
    print()
print()
print(f'Number of distinct SMARTS: {len(set(smarts_to_smis.keys()))}')
print(f'Number of distinct SMILES: {len(set([j for i in smarts_to_smis.values() for j in i]))}')

In [ ]:
for bit in submol_freq_distrib:
    print(f'Bit ID: {bit}')
    for submol in submol_freq_distrib[bit]:
        print(f'Substructure (SMARTS): {submol}')
        print(f'Frequency: {submol_freq_distrib[bit][submol]}')
    print()

In [ ]:
ctrs = {bit: 0 for bit in bits_of_interest}
for b in bits_to_draw:
    mol_examples = list(examples_all[b[2]].values())[ctrs[b[2]]]
    ctrs[b[2]] += 1
    for ex_num in mol_examples:
        d = Draw.DrawMorganBit(b[1], b[2], b[3], whichExample=ex_num, aromaticColor=None, ringColor=None)
        d.save(pathify(f'twobits_triazine_SIRT2_FP-FFNN_random_seed_0_cpd_id_{b[0]}_bit_{b[2]}_example_{ex_num}.png'), 'PNG')

## Substructure analysis

### Number substructures

In [ ]:
# numbering correponds to ordering of bars from left to right on plot of substructure weights (see below)
ctr = 1
submol_to_id = {}
for bit in bits_of_interest:
    submols = sorted(submol_freq_distrib[bit].keys(), key = lambda x: submol_freq_distrib[bit][x], reverse=True)
    for s in submols:
        submol_to_id[s] = ctr
        ctr += 1

In [ ]:
hkl.dump(submol_to_id, 'twobits_submol_to_id_triazine_SIRT2_FP-FFNN_random_seed_0.hkl', mode='w')

In [ ]:
# submol_to_id = hkl.load('twobits_submol_to_id_triazine_SIRT2_FP-FFNN_random_seed_0.hkl')

In [ ]:
for item in sorted(submol_to_id.items(), key=lambda x: x[1]):
    print(item[1])
    print(item[0])
    print(np.squeeze(smarts_to_smis[item[0]]))
    print()

### Calculate substructure weights

In [ ]:
substruct_to_weight = {}
for submol in submol_to_cpd_indices:
    bit = int(np.squeeze(submol_to_bit[submol]))
    substruct_to_weight[submol] = GetWeightsForSubstructure(bit_to_weights[bit][0], list(set(submol_to_cpd_indices[submol])))[1]

In [ ]:
for item in sorted(substruct_to_weight.items(), key=lambda x: submol_to_id[x[0]]):
    print(f'Substructure (SMARTS): {item[0]}')
    print()
    print(f'Weight: {item[1]}')
    print()
    print()

### Plot substructure weights

In [ ]:
substructs = sorted(substruct_to_weight.keys(), key=lambda s: submol_to_id[s])
weights = [substruct_to_weight[s] for s in substructs]
bars = []
for bit in bits_of_interest:
    _bars = []
    substruct_smarts_for_bit = submol_freq_distrib[bit].keys()
    for s in substructs:
        if s in substruct_smarts_for_bit:
            _bars.append(substruct_to_weight[s])
    bars.append(_bars)

barWidth = 2
offset = 2.2
space = 4.5

pos = len(bars[0])*barWidth / 2

In [ ]:
r1 = [pos - 0.5*barWidth - 0.5*(offset-barWidth), pos + 0.5*barWidth + 0.5*(offset-barWidth)]
pos += space + 2*barWidth + (offset-barWidth)
r2 = [pos - barWidth - (offset-barWidth), pos, pos + barWidth + (offset-barWidth)]
pos += space + 2*barWidth + (offset-barWidth)
r3 = [pos - barWidth - (offset-barWidth), pos, pos + barWidth + (offset-barWidth)]
pos += space + 2*barWidth + (offset-barWidth)
r4 = [pos - 1.5*barWidth - 1.5*(offset-barWidth), pos - 0.5*barWidth - 0.5*(offset-barWidth), 
            pos + 0.5*barWidth + 0.5*(offset-barWidth), pos + 1.5*barWidth + 1.5*(offset-barWidth)]
pos += space + 2*barWidth + (offset-barWidth)
r5 = [pos - 0.5*barWidth - 0.5*(offset-barWidth), pos + 0.5*barWidth + 0.5*(offset-barWidth)]
pos += space + 2*barWidth + (offset-barWidth)
r6 = [pos]
pos += space + 2*barWidth + (offset-barWidth)
r7 = [pos]
pos += space + 2*barWidth + (offset-barWidth)
r8 = [pos - barWidth - (offset-barWidth), pos, pos + barWidth + (offset-barWidth)]
rs = [r1, r2, r3, r4, r5, r6, r7, r8]

In [ ]:
fig = plt.figure(figsize=(7, 4), dpi=300)

rects = []
for bit in range(len(bars)):
    for i in range(len(rs[bit])):
        rects.append(plt.bar(rs[bit][i], bars[bit][i], color='#1f77b4', width=barWidth, zorder=2)[0])

for rect in rects[:-5]:
    height = rect.get_height()
    plt.annotate('{:.3f}'.format(height),
                xy=(rect.get_x() + rect.get_width() / 2, height),
                xytext=(0, 3),  
                textcoords="offset points",
                ha='center', va='bottom',
                fontsize=4.5)
for rect in rects[-5:]:
    height = rect.get_height()
    plt.annotate('{:.3f}'.format(height),
                xy=(rect.get_x() + rect.get_width() / 2, height),
                xytext=(0, -8),  
                textcoords="offset points",
                ha='center', va='bottom',
                fontsize=4.5)

fig.canvas.draw()
ax = plt.gca()
ax.grid(zorder=1)
ax.set_xlabel('bit ID', fontsize=9)
ax.set_ylabel('substructure weight', fontsize=9)
ax.tick_params(labelsize=9)
ax.set_xticks([((len(bars[0])*barWidth / 2) + i*(space + 2*barWidth + (offset-barWidth))) for i in range(len(bars))])
ax.set_xticklabels([str(bit_id) for bit_id in bits_of_interest], ha='center')
ax.tick_params(axis='x', length=0)
plt.tight_layout()
plt.savefig(pathify(f'twobits_substructure_weights_triazine_SIRT2_FP-FFNN_random_seed_0.png'))
plt.show()